In [ ]:
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
import pickle
import seaborn as sns
from matplotlib import rcParams
from matplotlib.ticker import MaxNLocator
import networkx as nx
from matplotlib import font_manager
from matplotlib.lines import Line2D

In [ ]:
# 设置全局字体为 Times New Roman
rcParams['font.family'] = 'serif'
rcParams['font.serif'] = ['Times New Roman']
rcParams['mathtext.fontset'] = 'custom'
rcParams['mathtext.rm'] = 'Times New Roman'
rcParams['mathtext.it'] = 'Times New Roman:italic'
rcParams['mathtext.bf'] = 'Times New Roman:bold'

In [ ]:
# 指定字体
font_path = "C:/Windows/Fonts/simsun.ttc"  # SimSun字体路径
font_prop = font_manager.FontProperties(fname=font_path)

plt.rcParams["font.sans-serif"] = [font_prop.get_name()]
plt.rcParams["axes.unicode_minus"] = False  # 解决负号显示问题
plt.rcParams["font.size"] = 10.5  # 设置五号字体大小

In [ ]:
# 设置xtick和ytick的方向：in、out、inout
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'

## 1.提取每个窗口下每组参数的初始观点值

In [ ]:
# 定义日志文件路径
# log_file = '../python/Multi_DySAT-HK/logs/log_danci_l1.txt'
log_file = '../python/Multi_DySAT-HK/对比实验/log_hk_l0.txt'

# 用于存储最终意见数据的嵌套字典
initial_opinions_data = {}

# 定义正则表达式
opinions_pattern = r'Window: (\d+)-(\d+), Initial Opinions: \{([^}]+)\}'

# 读取日志文件并处理每一行
with open(log_file, 'r') as file:
    lines = file.readlines()
    for line in lines:
        match = re.search(opinions_pattern, line)
        if match:
            start_window = int(match.group(1))
            end_window = int(match.group(2))
            opinions_str = match.group(3)
            
            # 解析 Opinions 字符串
            opinions = {}
            for item in opinions_str.split(','):
                node, opinion = item.split(':')
                opinions[int(node.strip())] = float(opinion.strip())
            
            # 将数据存储到嵌套字典中
            if start_window not in initial_opinions_data:
                initial_opinions_data[start_window] = {}
            initial_opinions_data[start_window] = opinions

In [ ]:
initial_opinions_data

## 2.提取每个窗口下每组参数的稳定观点值

In [ ]:
# 定义日志文件路径
# log_file = '../python/Multi_DySAT-HK/logs/log_danci_l1.txt'
log_file = '../python/Multi_DySAT-HK/对比实验/log_hk_l0.txt'

# 用于存储最终意见数据的嵌套字典
final_opinions_data = {}

# 定义正则表达式
opinions_pattern = r'Window: (\d+)-(\d+), Parameters: \(([^,]+), ([^,]+), ([^)]+)\), Final Opinions: \{([^}]+)\}'

# 读取日志文件并处理每一行
with open(log_file, 'r') as file:
    lines = file.readlines()
    for line in lines:
        match = re.search(opinions_pattern, line)
        if match:
            start_window = int(match.group(1))
            end_window = int(match.group(2))
            param1 = float(match.group(3))
            param2 = float(match.group(4))
            param3 = float(match.group(5))
            opinions_str = match.group(6)
            
            # 解析 Opinions 字符串
            opinions = {}
            for item in opinions_str.split(','):
                node, opinion = item.split(':')
                opinions[int(node.strip())] = float(opinion.strip())
            
            # 将数据存储到嵌套字典中
            if start_window not in final_opinions_data:
                final_opinions_data[start_window] = {}
            final_opinions_data[start_window][(param1, param2, param3)] = opinions

In [ ]:
final_opinions_data

## 3.用于存储各个窗口下各节点观点差异的绝对值

In [ ]:
opinion_differences = {}

# 遍历每个窗口
for window in initial_opinions_data:
    initial_opinions = initial_opinions_data[window]
    if window not in final_opinions_data:
        continue

    # 遍历每组参数组合
    for params in final_opinions_data[window]:
        final_opinions = final_opinions_data[window][params]

        # 计算观点差异的绝对值
        differences = {}
        for node in initial_opinions:
            if node in final_opinions:
                difference = abs(final_opinions[node] - initial_opinions[node])
                differences[node] = difference

        # 将数据存储到嵌套字典中
        if window not in opinion_differences:
            opinion_differences[window] = {}
        opinion_differences[window][params] = differences


In [ ]:
opinion_differences

In [ ]:
# 用于存储每个窗口和参数组合的最大观点差异值
max_opinion_differences = {}

# 遍历 opinion_differences 计算每个窗口和参数组合的最大差异值
for window, params_dict in opinion_differences.items():
    max_differences_for_window = {}
    for params, differences in params_dict.items():
        max_difference = max(differences.values())
        max_differences_for_window[params] = max_difference
    
    max_opinion_differences[window] = max_differences_for_window


In [ ]:
max_opinion_differences

In [ ]:
# 找到所有窗口和参数组合中的最大值
overall_max_difference = None

for window, params_dict in max_opinion_differences.items():
    for params, max_difference in params_dict.items():
        if overall_max_difference is None or max_difference > overall_max_difference:
            overall_max_difference = max_difference
print(overall_max_difference)

## 4.真实标签

In [ ]:
def load_snapshots(file_path):
    with open(file_path, 'rb') as f:
        snapshots = pickle.load(f)
    return snapshots

In [ ]:
# 加载数据
dataset_path = '../python/Multi_DySAT-HK/网络数据/graph_snapshots_l1.pkl'
graphs = load_snapshots(dataset_path)

window_labels_dict = {} 
# 外层循环遍历窗口
for start_idx in range(0, 31, 1):
    end_idx = start_idx + 5
    if end_idx > len(graphs):
        end_idx = len(graphs)

    # 选择当前窗口内的图数据
    window_graphs = graphs[start_idx:end_idx]

    active_nodes = [node for i, node in enumerate(window_graphs[3].nodes()) if
                    window_graphs[3].nodes[node]['category'] == 1]

    true_labels = [int(window_graphs[4].nodes[i]['category']) for i in active_nodes]
    
    # 将当前窗口的 true_labels 存储到字典中
    window_labels_dict[start_idx] = true_labels


In [ ]:
import pandas as pd

# 加载数据
dataset_path = '../python/Multi_DySAT-HK/网络数据/graph_snapshots_l1.pkl'
graphs = load_snapshots(dataset_path)

# 初始化字典，存储每个窗口的节点ID及其对应的标签和边
window_labels_dict = {}
window_edges_dict = {}

# 外层循环遍历窗口
for start_idx in range(0, 4, 1):
    end_idx = start_idx + 5
    if end_idx > len(graphs):
        end_idx = len(graphs)

    # 选择当前窗口内的图数据
    window_graphs = graphs[start_idx:end_idx]

    # 获取当前窗口第一张快照的节点ID和标签
    node_ids = list(window_graphs[0].nodes())
    true_labels = [int(window_graphs[0].nodes[i]['category']) for i in node_ids]

    # 获取当前窗口第一张快照的边 (边为节点对)
    edges = list(window_graphs[0].edges())

    # 将节点ID和标签保存到字典中，键为窗口编号，值为DataFrame
    window_labels_dict[start_idx] = pd.DataFrame({
        'Id': node_ids,
        'Label': true_labels
    })

    # 将边保存到字典中，键为窗口编号，值为包含边的DataFrame
    window_edges_dict[start_idx] = pd.DataFrame(edges, columns=['Source', 'Target'])

# 保存每个窗口的节点ID和标签为 CSV 文件
for window_idx, df in window_labels_dict.items():
    csv_path = f'D:/研究生/毕业论文/代码/网络构建/直播平台网络模型/jupyter/网络数据/l1_label_{window_idx}.csv'
    df.to_csv(csv_path, index=False)
    print(f"Window {window_idx} node labels saved to {csv_path}")

# 保存每个窗口的边为 CSV 文件
for window_idx, df in window_edges_dict.items():
    csv_path = f'D:/研究生/毕业论文/代码/网络构建/直播平台网络模型/jupyter/网络数据/l1_edges_{window_idx}.csv'
    df.to_csv(csv_path, index=False)
    print(f"Window {window_idx} edges saved to {csv_path}")



In [ ]:
window_labels_dict

In [ ]:
# 加载数据
dataset_path = '../python/Multi_DySAT-HK/网络数据/graph_snapshots_l1_test.pkl'
graphs = load_snapshots(dataset_path)

window_labels_dict = {} 
label_ratios = {}  # 存储每个start_idx中 true_labels为1的比例
# 外层循环遍历窗口
for start_idx in range(0, 31, 1):
    end_idx = start_idx + 5
    if end_idx > len(graphs):
        end_idx = len(graphs)

    # 选择当前窗口内的图数据
    window_graphs = graphs[start_idx:end_idx]

    active_nodes = [node for i, node in enumerate(window_graphs[3].nodes()) if
                    window_graphs[3].nodes[node]['category'] == 1]

    true_labels = [int(window_graphs[4].nodes[i]['category']) for i in active_nodes]
    
    # 将当前窗口的 true_labels 存储到字典中
    window_labels_dict[start_idx] = true_labels
    
    # 计算 true_labels 中等于1的比例
    if len(true_labels) > 0:
        ratio = true_labels.count(1) / len(true_labels)
    else:
        ratio = 0  # 避免除以0的情况
    
    # 将比例存储到字典中
    label_ratios[start_idx] = ratio

# 输出每个窗口的比例
for start_idx, ratio in label_ratios.items():
    print(f"start_idx: {start_idx}, true_labels为1的比例: {ratio:.2f}")


## 5.得到最优参数组合

In [ ]:
# 定义阈值的范围
thresholds = np.arange(0.1, 0.55, step=0.1)

# 存储每个窗口下最佳阈值和对应的评估结果
best_thresholds_and_results = {}

# 遍历每个窗口
for start_window, changes_data in opinion_differences.items():
    
    # 提取真实标签
    true_labels = window_labels_dict[start_window]

    # 存储每个参数组合的最佳阈值和对应的最佳 F1 分数
    best_thresholds_and_f1 = []

    # 遍历每个参数组合
    for params, opinion_changes_dict in changes_data.items():
        # 存储当前参数组合的最佳阈值和对应的最佳 F1 分数
        best_f1_score = 0
        best_precision = 0
        best_recall = 0
        best_threshold = None

        # 遍历每个阈值
        for threshold in thresholds:
            # 存储当前阈值下的预测标签
            binary_labels_threshold = {node_id: 0 if change > threshold else 1 for node_id, change in opinion_changes_dict.items()}

            # 检查预测标签长度与真实标签长度是否一致
            if len(true_labels) != len(binary_labels_threshold):
                continue

            # 计算 F1 分数、recall 和 precision
            precision, recall, f1, _ = precision_recall_fscore_support(true_labels, list(binary_labels_threshold.values()), average='weighted', zero_division=0)

            # 更新最佳 F1 分数和对应的阈值
            if f1 > best_f1_score:
                best_f1_score = f1
                best_precision = precision
                best_recall = recall
                best_threshold = threshold

        # 在每个参数组合下，存储最佳阈值和对应的最佳 F1 分数
        best_thresholds_and_f1.append({
            'parameters': params,
            'best_threshold': best_threshold,
            'best_f1_score': best_f1_score,
            'precision': best_precision,
            'recall': best_recall
        })

    # 在每个窗口下，找到具有最佳 F1 分数的参数组合和阈值
    best_result = max(best_thresholds_and_f1, key=lambda x: x['best_f1_score'])

    # 存储最佳阈值和评估结果
    best_thresholds_and_results[start_window] = {
        'best_parameters': best_result['parameters'],
        'best_threshold': best_result['best_threshold'],
        'precision': best_result['precision'],
        'recall': best_result['recall'],
        'f1_score': best_result['best_f1_score']
    }


In [ ]:
from sklearn.metrics import accuracy_score

# 定义阈值的范围
thresholds = np.arange(0.1, 0.55, step=0.1)

# 存储每个窗口下最佳阈值和对应的评估结果
best_thresholds_and_results = {}

# 遍历每个窗口
for start_window, changes_data in opinion_differences.items():
    
    # 提取真实标签
    true_labels = window_labels_dict[start_window]

    # 存储每个参数组合的最佳阈值和对应的最佳 accuracy
    best_thresholds_and_accuracy = []

    # 遍历每个参数组合
    for params, opinion_changes_dict in changes_data.items():
        # 存储当前参数组合的最佳阈值和对应的最佳 accuracy
        best_accuracy = 0
        best_threshold = None

        # 遍历每个阈值
        for threshold in thresholds:
            # 存储当前阈值下的预测标签
            binary_labels_threshold = {node_id: 0 if change > threshold else 1 for node_id, change in opinion_changes_dict.items()}

            # 检查预测标签长度与真实标签长度是否一致
            if len(true_labels) != len(binary_labels_threshold):
                continue

            # 计算 accuracy
            accuracy = accuracy_score(true_labels, list(binary_labels_threshold.values()))

            # 更新最佳 accuracy 和对应的阈值
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_threshold = threshold

        # 在每个参数组合下，存储最佳阈值和对应的最佳 accuracy
        best_thresholds_and_accuracy.append({
            'parameters': params,
            'best_threshold': best_threshold,
            'best_accuracy': best_accuracy
        })

    # 在每个窗口下，找到具有最佳 accuracy 的参数组合和阈值
    best_result = max(best_thresholds_and_accuracy, key=lambda x: x['best_accuracy'])

    # 存储最佳阈值和评估结果
    best_thresholds_and_results[start_window] = {
        'best_parameters': best_result['parameters'],
        'best_threshold': best_result['best_threshold'],
        'accuracy': best_result['best_accuracy']
    }

# 输出每个窗口下的最佳结果
for start_window, result in best_thresholds_and_results.items():
    print(f"窗口 {start_window}: 最佳参数组合: {result['best_parameters']}, "
          f"最佳阈值: {result['best_threshold']}, "
          f"正确率: {result['accuracy']:.2f}")


In [ ]:
best_thresholds_and_results

In [ ]:
# 创建一个空的DataFrame
result = []
# 遍历每个窗口
for window_key, data_list in best_thresholds_and_results.items():

    best_parameters = data_list['best_parameters']
    best_threshold = data_list['best_threshold']
    result.append({
        'similarity_threshold': best_parameters[0],
        'state_diff_weight': best_parameters[1],
        'update_speed': best_parameters[2],
        'best_threshold': best_threshold,
    })

# 保存为CSV文件
df = pd.DataFrame(result)
csv_path = 'best_parameters_and_thresholds_l1.csv'
df.to_csv(csv_path, index=False)

# 打印路径以供参考
print(f"CSV文件已保存至: {csv_path}")


## 6.可视化

### 6.1.最优参数

In [ ]:
# 读取数据
csv_path = 'best_parameters_and_thresholds_l1.csv'
df = pd.read_csv(csv_path)

# 提取数据
windows = df.index
similarity_threshold = df['similarity_threshold']
state_diff_weight = df['state_diff_weight']
update_speed = df['update_speed']
best_threshold = df['best_threshold']

# 颜色定义
def addcolorplus(n):
    """生成 n 种颜色的调色板"""
    cmap = plt.get_cmap('tab20')  # 可以选择其他的颜色映射
    return cmap(np.linspace(0, 1, n))

idx = np.linspace(0, 19, 4).astype(int)  # 使用 20 种颜色的 colormap
C = addcolorplus(20)[idx, :]

# 创建堆叠折线图
fig, ax = plt.subplots(figsize=(12, 8), dpi=300)

# 绘制堆叠折线图
ax.stackplot(windows, similarity_threshold, state_diff_weight, update_speed, best_threshold,
             labels=['Opinion threshold', 'Confidence level', 'Update speed', 'Behavior threshold'],
             colors=C, alpha=0.7)

# 设置字体和字号
ax.set_xlabel('Window', fontsize=12, fontname='Times New Roman')
ax.set_ylabel('Value', fontsize=12, fontname='Times New Roman')
ax.legend(loc='best', fontsize=10)
ax.grid(True, linestyle='--', alpha=0.7)

# 设置背景颜色
fig.patch.set_facecolor('white')

# 调整布局
plt.tight_layout()

# # 保存图片
# plt.savefig('best_parameters_and_thresholds_stackedplot.pdf', format='pdf', dpi=300)

# 显示图形
plt.show()


### 6.2.观点差异

In [ ]:
# 提取每个窗口下最优参数下观点演化情况

# 日志文件路径
# log_file = '../python/Multi_DySAT-HK/logs/log_best_danci_l0.txt'
log_file = '../python/Multi_DySAT-HK/对比实验/log_hk_l1.txt'

# 用于匹配的正则表达式模式
log_pattern = re.compile(r"Window: (\d+)-(\d+), Iteration: (\d+), Opinions: \{(.+?)\}")

# 用于存储每个窗口下每次迭代的意见的嵌套字典
opinion_data = {}

# 读取日志文件并处理每一行
with open(log_file, 'r') as file:
    lines = file.readlines()
    for line in lines:
        match = re.search(log_pattern, line)
        if match:
            start_window = int(match.group(1))
            end_window = int(match.group(2))
            iteration = int(match.group(3))
            opinions_str = match.group(4)
            
            # 解析 Opinions 字符串
            opinions = {}
            for item in opinions_str.split(','):
                node, opinion = item.split(':')
                opinions[int(node.strip())] = float(opinion.strip())
            
            # 将数据存储到嵌套字典中
            if start_window not in opinion_data:
                opinion_data[start_window] = {}
            if iteration not in opinion_data[start_window]:
                opinion_data[start_window][iteration] = opinions

In [ ]:
opinion_data

In [ ]:
from matplotlib.lines import Line2D

# 设置窗口的总数和子图的规格
num_windows = 31
num_rows = 11
num_cols = 3

# 读取 CSV 文件
csv_path = 'best_parameters_and_thresholds_l1.csv'
df = pd.read_csv(csv_path)

# 提取窗口和参数
windows = df.index
similarity_threshold = df['similarity_threshold']
state_diff_weight = df['state_diff_weight']
update_speed = df['update_speed']
best_threshold = df['best_threshold']

# 进行循环之前，创建一个空的图例列表
legend_elements = []


# 创建一个3x10的子图网格
fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(12, 18))
axes = axes.flatten()  # 将子图的二维数组展平成一维数组，方便索引

# 遍历每个窗口，并在对应的子图上绘制
for i in range(num_windows):
    ax = axes[i]
    
    # 提取真实标签
    true_labels = window_labels_dict[i]
    
    # 提取当前窗口的意见数据
    current_opinions = opinion_data.get(i, {})

    iterations = []
    opinions = []

    # 各个迭代次数对应的观点值
    for iteration, opinions_data in current_opinions.items():
        iterations.append(iteration)
        # 存储观点数据
        opinions.append(opinions_data)

    # 绘制观点演化图
    for node, values in opinions[0].items():
        node_opinions = [opinions_data[node] for opinions_data in opinions]

        # 获取节点的位置索引
        node_index = list(opinions[0].keys()).index(node)

        # 根据行为值选择颜色
        color = '#6390eb' if true_labels[node_index] == 1 else '#fe6a6a'  # 根据位置索引选择颜色

        # 绘制观点演化图，并使用不同颜色
        ax.plot(iterations, node_opinions, color=color, linestyle='-')

        # 添加图例元素
        label = '保持在线的节点' if true_labels[node_index] == 1 else '在线转化为离线的节点'
        if label not in [leg.get_label() for leg in legend_elements]:
            legend_elements.append(Line2D([0], [0], color=color, label=label))

    # 在子图底部添加参数信息
    ax.text(0.5, -0.3, f'({i+1})  (δ, ε, λ) = ({state_diff_weight[i]},{similarity_threshold[i]}, {update_speed[i]})',
        ha='center', va='center', fontsize=12, fontproperties=font_prop, transform=ax.transAxes)
    
    # 仅在第一列添加 y 标签
    if i % num_cols == 0:
        ax.set_ylabel('观点', fontsize=12, fontproperties=font_prop)

    if i >= (num_rows - 1) * num_cols:
        ax.set_xlabel('时间', fontsize=12, fontproperties=font_prop,labelpad=20)

    # 设置坐标轴刻度标签字体大小
    ax.tick_params(axis='both', which='major', labelsize=10)
    
    # 使用整数坐标
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))

# 添加图例
fig.legend(handles=legend_elements, loc='lower right', fontsize=12, bbox_to_anchor=(0.95, 0.05),ncol=2)

# 如果窗口数小于子图总数，隐藏多余的子图
if num_windows < num_rows * num_cols:
    for j in range(num_windows, num_rows * num_cols):
        axes[j].axis('off')

# 调整布局
plt.tight_layout()

# 保存图形为 PDF
plt.savefig('D:/研究生/论文/毕业论文/草稿/图片/opinions_best_l1.svg')

plt.show()


In [ ]:
from matplotlib.lines import Line2D

# 设置窗口的总数和子图的规格
num_windows = 31
num_rows = 11
num_cols = 3

# 读取 CSV 文件
csv_path = 'best_parameters_and_thresholds_l0.csv'
df = pd.read_csv(csv_path)

# 提取窗口和参数
windows = df.index
similarity_threshold = df['similarity_threshold']
state_diff_weight = df['state_diff_weight']
update_speed = df['update_speed']
best_threshold = df['best_threshold']

# 进行循环之前，创建一个空的图例列表
legend_elements = []


# 创建一个3x10的子图网格
fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(12, 18))
axes = axes.flatten()  # 将子图的二维数组展平成一维数组，方便索引

# 遍历每个窗口，并在对应的子图上绘制
for i in range(num_windows):
    ax = axes[i]
    
    # 提取真实标签
    true_labels = window_labels_dict[i]
    
    # 提取当前窗口的意见数据
    current_opinions = opinion_data.get(i, {})

    iterations = []
    opinions = []

    # 各个迭代次数对应的观点值
    for iteration, opinions_data in current_opinions.items():
        iterations.append(iteration)
        # 存储观点数据
        opinions.append(opinions_data)

    # 绘制观点演化图
    for node, values in opinions[0].items():
        node_opinions = [opinions_data[node] for opinions_data in opinions]

        # 获取节点的位置索引
        node_index = list(opinions[0].keys()).index(node)

        # 根据行为值选择颜色
        color = '#6390eb' if true_labels[node_index] == 1 else '#fe6a6a'  # 根据位置索引选择颜色

        # 绘制观点演化图，并使用不同颜色
        ax.plot(iterations, node_opinions, color=color, linestyle='-')

        # 添加图例元素
        label = 'Node that Stays Online' if true_labels[node_index] == 1 else 'Node that Switches from Online to Offline'
        if label not in [leg.get_label() for leg in legend_elements]:
            legend_elements.append(Line2D([0], [0], color=color, label=label))

    # 在子图底部添加参数信息
    ax.text(0.5, -0.3, f'({i+1})  (δ, ε, λ) = ({state_diff_weight[i]},{similarity_threshold[i]}, {update_speed[i]})',
        ha='center', va='center', fontsize=12, transform=ax.transAxes)
    
    # 仅在第一列添加 y 标签
    if i % num_cols == 0:
        ax.set_ylabel('Opinion', fontsize=12)

    if i >= (num_rows - 1) * num_cols:
        ax.set_xlabel('Time', fontsize=12,labelpad=20)

    # 设置坐标轴刻度标签字体大小
    ax.tick_params(axis='both', which='major', labelsize=10)
    
    # 使用整数坐标
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))

# 添加图例
fig.legend(handles=legend_elements, loc='lower right', fontsize=12, bbox_to_anchor=(0.95, 0.05),ncol=2)

# 如果窗口数小于子图总数，隐藏多余的子图
if num_windows < num_rows * num_cols:
    for j in range(num_windows, num_rows * num_cols):
        axes[j].axis('off')

# 调整布局
plt.tight_layout()

# 保存图形为 PDF
plt.savefig('D:/研究生/论文/期刊论文/小论文1/草稿/图片/opinions_best_l0.eps')

plt.show()


In [ ]:
# 可视化每个窗口的观点变动值

# 设置窗口的总数和子图的规格
num_windows = 29
num_rows = 7
num_cols = 4

# 创建一个7x4的子图网格
fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(20, 15))
axes = axes.flatten()  # 将子图的二维数组展平成一维数组，方便索引

# 遍历每个窗口，并在对应的子图上绘制
for i in range(num_windows - 1):  # 最后一个窗口不进行绘制
    ax = axes[i]
    
    # 提取真实标签
    true_labels = window_labels_dict[i]
    
    # 提取当前窗口的意见数据
    current_opinions = opinion_data.get(i, {})

    iterations = []
    opinions = []

    # 各个迭代次数对应的观点值
    for iteration, opinions_data in current_opinions.items():
        iterations.append(iteration)
        # 存储观点数据
        opinions.append(opinions_data)

    # 绘制观点变动情况
    for node, values in opinions[0].items():
        node_opinions = [opinions_data[node] for opinions_data in opinions]
        
        # 计算每次迭代的观点变动值
        opinion_changes = [0]  # 初始化变动值，第一次迭代没有变化值
        for k in range(1, len(node_opinions)):
            opinion_changes.append(node_opinions[k] - node_opinions[k-1])
        
        # 获取节点的位置索引
        node_index = list(opinions[0].keys()).index(node)

        # 根据行为值选择颜色
        color = 'red' if true_labels[node_index] == 1 else 'blue'  # 根据位置索引选择颜色

        # 绘制观点变动情况，并使用不同颜色
        ax.plot(iterations, opinion_changes, color=color, linestyle='-')

    # 设置标题和标签
    ax.set_xlabel('Iteration')
    ax.set_ylabel('Opinion Change')

# 隐藏最后一个子图（如果有的话）
if num_windows <= num_rows * num_cols:
    axes[-1].axis('off')

# 调整布局
plt.tight_layout()
plt.show()


In [ ]:
# 可视化每个窗口的观点变动值（相对于初始观点）

# 设置窗口的总数和子图的规格
num_windows = 29
num_rows = 7
num_cols = 4

# 创建一个7x4的子图网格
fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(20, 15))
axes = axes.flatten()  # 将子图的二维数组展平成一维数组，方便索引

# 遍历每个窗口，并在对应的子图上绘制
for i in range(num_windows - 1):  # 最后一个窗口不进行绘制
    ax = axes[i]
    
    # 提取真实标签
    true_labels = window_labels_dict[i]
    
    # 提取当前窗口的意见数据
    current_opinions = opinion_data.get(i, {})

    iterations = []
    opinions = []

    # 各个迭代次数对应的观点值
    for iteration, opinions_data in current_opinions.items():
        iterations.append(iteration)
        # 存储观点数据
        opinions.append(opinions_data)

    # 绘制观点变动情况
    for node, values in opinions[0].items():
        node_opinions = [opinions_data[node] for opinions_data in opinions]
        
        # 计算每次迭代相对于初始迭代的观点变动值
        initial_opinion = node_opinions[0]
        opinion_changes = [opinion - initial_opinion for opinion in node_opinions]
        
        # 获取节点的位置索引
        node_index = list(opinions[0].keys()).index(node)

        # 根据行为值选择颜色
        color = 'red' if true_labels[node_index] == 1 else 'blue'  # 根据位置索引选择颜色

        # 绘制观点变动情况，并使用不同颜色
        ax.plot(iterations, opinion_changes, color=color, linestyle='-')

    # 设置标题和标签
    ax.set_xlabel('Iteration')
    ax.set_ylabel('Opinion Change')

# 隐藏最后一个子图（如果有的话）
if num_windows <= num_rows * num_cols:
    axes[-1].axis('off')

# 调整布局
plt.tight_layout()
plt.show()


### 6.3.单次最优F1

In [ ]:
# 使用 Elsevier 格式的字体
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']

# 创建图表
plt.figure(figsize=(10, 6))

# 使用 ieee 风格
with plt.style.context(['ieee', 'no-latex']):
    # 获取窗口范围和评估结果
    window_ranges = [key for key in best_thresholds_and_results]
    f1_scores = [best_thresholds_and_results[key]['f1_score'] for key in best_thresholds_and_results]
    recall_scores = [best_thresholds_and_results[key]['recall'] for key in best_thresholds_and_results]
    precision_scores = [best_thresholds_and_results[key]['precision'] for key in best_thresholds_and_results]

    # 使用晶蓝色
    plt.plot(window_ranges, f1_scores, marker='o', linestyle='-', label='BsetBset F1')
    plt.plot(window_ranges, recall_scores, marker='o', linestyle='-', label='BsetBset Recall')
    plt.plot(window_ranges, precision_scores, marker='o', linestyle='-', label='Bset Precision')

    plt.xlabel('Window Range', fontsize=12)  # 增加标签间距
    plt.ylabel('Score', fontsize=12)  # 增加标签间距

    # plt.title('Bset F1, Bset Recall, and Bset Precision for Each Window', fontsize=14)  # 增大标题字号

    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    
    plt.legend(loc='best', fontsize=10)  # 设置图例字号和放置位置为最佳

    plt.tight_layout()
    plt.show()
    

### 6.4.多次最优F1

In [ ]:
# 提取每个窗口下每个种子数对应的初始观点
# 日志文件路径
# log_file = '../python/Multi_DySAT-HK/logs/log_repeat-100_l1.txt'
log_file = '../python/Multi_DySAT-HK/对比实验/log_hk_l1.txt'

# 用于存储最终意见数据的嵌套字典
initial_opinions_data = {}

# 定义正则表达式
opinions_pattern = r'Window: (\d+)-(\d+), Seed: (\d+), Initial Opinions: \{([^}]+)\}'

# 读取日志文件并处理每一行
with open(log_file, 'r') as file:
    lines = file.readlines()
    for line in lines:
        match = re.search(opinions_pattern, line)
        if match:
            start_window = int(match.group(1))
            end_window = int(match.group(2))
            seed = int(match.group(3))
            opinions_str = match.group(4)
            
            # 解析 Opinions 字符串
            opinions = {}
            for item in opinions_str.split(','):
                node, opinion = item.split(':')
                opinions[int(node.strip())] = float(opinion.strip())
            
            # 将数据存储到嵌套字典中
            if start_window not in initial_opinions_data:
                initial_opinions_data[start_window] = {}
            initial_opinions_data[start_window][seed] = opinions

In [ ]:
# 提取每个窗口下每个种子数对应的最终观点

# 日志文件路径
# log_file = '../python/Multi_DySAT-HK/logs/log_repeat-100_l1.txt'
log_file = '../python/Multi_DySAT-HK/对比实验/log_hk_l1.txt'

# 用于存储最终意见数据的嵌套字典
final_opinions_data = {}

# 定义正则表达式
opinions_pattern = r'Window: (\d+)-(\d+), Seed: (\d+), Final Opinions: \{([^}]+)\}'

# 读取日志文件并处理每一行
with open(log_file, 'r') as file:
    lines = file.readlines()
    for line in lines:
        match = re.search(opinions_pattern, line)
        if match:
            start_window = int(match.group(1))
            end_window = int(match.group(2))
            seed = int(match.group(3))
            opinions_str = match.group(4)
            
            # 解析 Opinions 字符串
            opinions = {}
            for item in opinions_str.split(','):
                node, opinion = item.split(':')
                opinions[int(node.strip())] = float(opinion.strip())
            
            # 将数据存储到嵌套字典中
            if start_window not in final_opinions_data:
                final_opinions_data[start_window] = {}
            final_opinions_data[start_window][seed] = opinions

In [ ]:
# 存储每个窗口下各个节点的观点差异
opinion_differences = {}

# 遍历每个窗口
for window in initial_opinions_data:
    if window not in final_opinions_data:
        continue

    # 遍历每组参数组合
    for seed in final_opinions_data[window]:
        initial_opinions = initial_opinions_data[window][seed]
        final_opinions = final_opinions_data[window][seed]

        # 计算观点差异的绝对值
        differences = {}
        for node in initial_opinions:
            if node in final_opinions:
                difference = abs(final_opinions[node] - initial_opinions[node])
                differences[node] = difference

        # 将数据存储到嵌套字典中
        if window not in opinion_differences:
            opinion_differences[window] = {}
        opinion_differences[window][seed] = differences

In [ ]:
# 计算每个窗口100次实验的平均评估指标值

import pandas as pd
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

# 读取保存最优参数的 CSV 文件
best_params_df = pd.read_csv('best_parameters_and_thresholds_l0.csv')

# 存储每个窗口下的评估结果
best_results = {}

# 遍历每个窗口
for start_window, changes_data in opinion_differences.items():

    # 获取当前窗口下的最优参数组合
    row = best_params_df.iloc[start_window]
    threshold = row['best_threshold']

    # 提取真实标签
    true_labels = window_labels_dict[start_window]

    # 存储每个参数组合的最佳 F1 分数
    f1_scores = []
    precisions = []
    recalls = []

    # 遍历每个参数组合
    for seed, opinion_changes_dict in changes_data.items():
        # 存储当前阈值下的预测标签
        binary_labels_threshold = {node_id: 0 if change > threshold else 1 for node_id, change in opinion_changes_dict.items()}

        # 检查预测标签长度与真实标签长度是否一致
        if len(true_labels) != len(binary_labels_threshold):
            continue

        # 计算 F1 分数、recall 和 precision
        precision, recall, f1, _ = precision_recall_fscore_support(true_labels, list(binary_labels_threshold.values()), average='weighted', zero_division=0)

        # 将分数添加到列表中
        f1_scores.append(f1)
        precisions.append(precision)
        recalls.append(recall)

    # 计算平均值
    avg_f1_score = np.mean(f1_scores)
    avg_precision = np.mean(precisions)
    avg_recall = np.mean(recalls)

    # 计算标准差
    std_f1_score = np.std(f1_scores)
    std_precision = np.std(precisions)
    std_recall = np.std(recalls)

    # 存储每个窗口的平均评估结果及其标准差
    best_results[start_window] = {
        'average_f1_score': avg_f1_score,
        'std_f1_score': std_f1_score,
        'average_precision': avg_precision,
        'std_precision': std_precision,
        'average_recall': avg_recall,
        'std_recall': std_recall
    }

# 将结果保存为 CSV 文件
result_df = pd.DataFrame(best_results).T  # 转置 DataFrame 使窗口编号为行索引
csv_path = 'window_evaluation_results_l1.csv'
result_df.to_csv(csv_path, index_label='window')

# 打印路径以供参考
print(f"CSV文件已保存至: {csv_path}")


In [ ]:
# 读取保存的评估结果 CSV 文件
result_df = pd.read_csv('window_evaluation_results_l1.csv', index_col='window')

lighter_blue = '#d0e1f9'   # 浅浅蓝色
lighter_red = '#ffcccc'    # 浅浅红色
lighter_cyan = '#ccf2f2'   # 浅浅青色

# 创建子图
fig, ax = plt.subplots(figsize=(8, 6))

# 绘制 Precision 及其标准差阴影
ax.plot(result_df.index + 1, result_df['average_precision'], marker='o', linestyle='-', color='#4169e0', label='Precision')
ax.fill_between(result_df.index + 1,
                result_df['average_precision'] - result_df['std_precision'],
                result_df['average_precision'] + result_df['std_precision'],
                color=lighter_blue)

# 绘制 Recall 及其标准差阴影
ax.plot(result_df.index + 1, result_df['average_recall'], marker='o', linestyle='-', color='#ff0000', label='Recall')
ax.fill_between(result_df.index + 1,
                result_df['average_recall'] - result_df['std_recall'],
                result_df['average_recall'] + result_df['std_recall'],
                color=lighter_red)

# 绘制 F1 分数及其标准差阴影
ax.plot(result_df.index + 1, result_df['average_f1_score'], marker='o', linestyle='-', color='#00cdd0', label=r'$F_1$')
ax.fill_between(result_df.index + 1,
                result_df['average_f1_score'] - result_df['std_f1_score'],
                result_df['average_f1_score'] + result_df['std_f1_score'],
                color=lighter_cyan)

# 设置标签和图例
ax.set_xlabel('Window', fontsize=16)
ax.set_ylabel('Score', fontsize=16)
ax.legend(loc='best', fontsize=16)
ax.grid(True, linestyle='--', alpha=0.7)

# 设置字体和字号
for label in ax.get_xticklabels() + ax.get_yticklabels():
    label.set_fontname('Times New Roman')
    label.set_fontsize(16)

# 设置背景颜色
fig.patch.set_facecolor('white')

# 调整布局
plt.tight_layout()

# # 保存图形为 PDF
# plt.savefig('D:/lunwen/social_network2023.7.17/caogao/Applied Soft Computing/figures/f1.eps', dpi=300)

# 显示图形
plt.show()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

# 读取保存最优参数的 CSV 文件
best_params_df = pd.read_csv('best_parameters_and_thresholds_l1.csv')

# 存储每个窗口下的评估结果
best_results = {}

# 遍历每个窗口
for start_window, changes_data in opinion_differences.items():

    # 获取当前窗口下的最优参数组合
    row = best_params_df.iloc[start_window]
    threshold = row['best_threshold']

    # 提取真实标签
    true_labels = window_labels_dict[start_window]

    # 存储每个参数组合的正确率
    accuracies = []

    # 遍历每个实验
    for seed, opinion_changes_dict in changes_data.items():
        # 存储当前阈值下的预测标签
        binary_labels_threshold = {node_id: 0 if change > threshold else 1 for node_id, change in opinion_changes_dict.items()}

        # 检查预测标签长度与真实标签长度是否一致
        if len(true_labels) != len(binary_labels_threshold):
            continue

        # 计算正确率
        accuracy = accuracy_score(true_labels, list(binary_labels_threshold.values()))

        # 将正确率添加到列表中
        accuracies.append(accuracy)

    # 计算平均正确率
    avg_accuracy = np.mean(accuracies)

    # 计算正确率的标准差
    std_accuracy = np.std(accuracies)

    # 存储每个窗口的平均正确率及其标准差
    best_results[start_window] = {
        'average_accuracy': avg_accuracy,
        'std_accuracy': std_accuracy
    }

# 将结果保存为 CSV 文件
result_df = pd.DataFrame(best_results).T  # 转置 DataFrame 使窗口编号为行索引
csv_path = 'hk_window_accuracy_results_l1.csv'
result_df.to_csv(csv_path, index_label='window')

# 打印路径以供参考
print(f"CSV文件已保存至: {csv_path}")


In [ ]:
# 读取保存的评估结果 CSV 文件
result_df_l0 = pd.read_csv('window_accuracy_results_l0.csv', index_col='window')
result_df_l1 = pd.read_csv('window_accuracy_results_l1.csv', index_col='window')

lighter_blue = '#d0e1f9'   # 浅浅蓝色
lighter_red = '#ffcccc'    # 浅浅红色

# 设置 Matplotlib 使用 ieee 样式
with plt.style.context(['ieee', 'no-latex']):

    # 创建子图
    fig, ax = plt.subplots(figsize=(8, 6))

    # 绘制 Precision 及其标准差阴影
    ax.plot(result_df_l0.index + 1, result_df_l0['average_accuracy'], marker='o', linestyle='-', color='#ff0000', label=r'$L_1$')
    ax.fill_between(result_df_l0.index + 1,
                    result_df_l0['average_accuracy'] - result_df_l0['std_accuracy'],
                    result_df_l0['average_accuracy'] + result_df_l0['std_accuracy'],
                    color=lighter_red)

    # 绘制 Recall 及其标准差阴影
    ax.plot(result_df_l1.index + 1, result_df_l1['average_accuracy'], marker='s', linestyle='-', color='#4169e0', label=r'$L_2$')
    ax.fill_between(result_df_l1.index + 1,
                    result_df_l1['average_accuracy'] - result_df_l1['std_accuracy'],
                    result_df_l1['average_accuracy'] + result_df_l1['std_accuracy'],
                    color=lighter_blue)

    # 设置标签和图例
    ax.set_xlabel('窗口(无单位)', fontproperties=font_prop,fontsize=14)
    ax.set_ylabel('准确率', fontproperties=font_prop,fontsize=14)
    ax.legend(loc='best', fontsize=14)
    ax.grid(True, linestyle='--', alpha=0.7)

    # 设置字体和字号
    for label in ax.get_xticklabels() + ax.get_yticklabels():
        label.set_fontname('Times New Roman')
        label.set_fontsize(14)

    # 设置背景颜色
    fig.patch.set_facecolor('white')

    # 调整布局
    plt.tight_layout()

    # 保存图形为 PDF
    plt.savefig('D:/研究生/毕业论文/草稿/图片/acc.svg')

    # 显示图形
    plt.show()

In [ ]:
# 读取保存的评估结果 CSV 文件
result_df_l0 = pd.read_csv('window_accuracy_results_l0.csv', index_col='window')
result_df_l1 = pd.read_csv('window_accuracy_results_l1.csv', index_col='window')

lighter_blue = '#d0e1f9'   # 浅浅蓝色
lighter_red = '#ffcccc'    # 浅浅红色

# 设置 Matplotlib 使用 ieee 样式
with plt.style.context(['ieee', 'no-latex']):

    # 创建子图
    fig, ax = plt.subplots(figsize=(8, 6))

    # 绘制 Precision 及其标准差阴影
    ax.plot(result_df_l0.index + 1, result_df_l0['average_accuracy'], marker='o', linestyle='-', color='#ff0000', label=r'$L_1$')
    ax.fill_between(result_df_l0.index + 1,
                    result_df_l0['average_accuracy'] - result_df_l0['std_accuracy'],
                    result_df_l0['average_accuracy'] + result_df_l0['std_accuracy'],
                    color=lighter_red)

    # 绘制 Recall 及其标准差阴影
    ax.plot(result_df_l1.index + 1, result_df_l1['average_accuracy'], marker='s', linestyle='-', color='#4169e0', label=r'$L_2$')
    ax.fill_between(result_df_l1.index + 1,
                    result_df_l1['average_accuracy'] - result_df_l1['std_accuracy'],
                    result_df_l1['average_accuracy'] + result_df_l1['std_accuracy'],
                    color=lighter_blue)

    # 设置标签和图例
    ax.set_xlabel('Window',fontsize=14)
    ax.set_ylabel('Accuracy',fontsize=14)
    ax.legend(loc='upper left', fontsize=14)
    ax.grid(True, linestyle='--', alpha=0.7)

    # 设置字体和字号
    for label in ax.get_xticklabels() + ax.get_yticklabels():
        label.set_fontname('Times New Roman')
        label.set_fontsize(14)

    # 设置背景颜色
    fig.patch.set_facecolor('white')

    # 调整布局
    plt.tight_layout()

    # 保存图形为 PDF
    plt.savefig('D:/研究生/论文/期刊论文/小论文1/草稿/图片/acc.eps')

    # 显示图形
    plt.show()

In [ ]:
# 读取保存的评估结果 CSV 文件
result_df_l0 = pd.read_csv('window_accuracy_results_l0.csv', index_col='window')
result_df_l1 = pd.read_csv('window_accuracy_results_l1.csv', index_col='window')
result_df_l0_hk = pd.read_csv('hk_window_accuracy_results_l0.csv', index_col='window')
result_df_l1_hk = pd.read_csv('hk_window_accuracy_results_l1.csv', index_col='window')

lighter_blue = '#d0e1f9'   # 浅浅蓝色
lighter_red = '#ffcccc'    # 浅浅红色

# 设置 Matplotlib 使用 ieee 样式
with plt.style.context(['ieee', 'no-latex']):

    # 创建子图
    fig, ax = plt.subplots(figsize=(8, 6))

    # 绘制 Precision 及其标准差阴影
    ax.plot(result_df_l0.index + 1, result_df_l0['average_accuracy'], marker='o', linestyle='-', color='#ff0000', label=r'RM-DySAT-BC')
    ax.fill_between(result_df_l0.index + 1,
                    result_df_l0['average_accuracy'] - result_df_l0['std_accuracy'],
                    result_df_l0['average_accuracy'] + result_df_l0['std_accuracy'],
                    color=lighter_red, alpha=0.6)
    
    ax.plot(result_df_l0_hk.index + 1, result_df_l0_hk['average_accuracy'], marker='s', linestyle='-', color='#4169e0', label=r'HK')
    ax.fill_between(result_df_l0_hk.index + 1,
                    result_df_l0_hk['average_accuracy'] - result_df_l0_hk['std_accuracy'],
                    result_df_l0_hk['average_accuracy'] + result_df_l0_hk['std_accuracy'],
                    color=lighter_blue, alpha=0.6)

    ax.set_ylim(0.35, 0.7)

    # 设置标签和图例
    ax.set_xlabel(r'(a) The window of $L_1$',fontsize=14)
    ax.set_ylabel('Accuracy',fontsize=14)
    ax.legend(loc='lower right', fontsize=12)
    ax.grid(True, linestyle='--', alpha=0.7)

    # 设置字体和字号
    for label in ax.get_xticklabels() + ax.get_yticklabels():
        label.set_fontname('Times New Roman')
        label.set_fontsize(14)

    # 设置背景颜色
    fig.patch.set_facecolor('white')

    # 调整布局
    plt.tight_layout()

    # 保存图形为 PDF
    plt.savefig('D:/研究生/论文/期刊论文/小论文1/新期刊/草稿/figures/acc_l0.pdf')

    # 显示图形
    plt.show()

In [ ]:
# 读取保存的评估结果 CSV 文件
result_df_l0 = pd.read_csv('window_accuracy_results_l0.csv', index_col='window')
result_df_l1 = pd.read_csv('window_accuracy_results_l1.csv', index_col='window')
result_df_l0_hk = pd.read_csv('hk_window_accuracy_results_l0.csv', index_col='window')
result_df_l1_hk = pd.read_csv('hk_window_accuracy_results_l1.csv', index_col='window')

lighter_blue = '#d0e1f9'   # 浅浅蓝色
lighter_red = '#ffcccc'    # 浅浅红色

# 设置 Matplotlib 使用 ieee 样式
with plt.style.context(['ieee', 'no-latex']):

    # 创建子图
    fig, ax = plt.subplots(figsize=(8, 6))

    # 绘制 Precision 及其标准差阴影
    ax.plot(result_df_l1.index + 1, result_df_l1['average_accuracy'], marker='o', linestyle='-', color='#ff0000', label=r'RM-DySAT-BC')
    ax.fill_between(result_df_l1.index + 1,
                    result_df_l1['average_accuracy'] - result_df_l1['std_accuracy'],
                    result_df_l1['average_accuracy'] + result_df_l1['std_accuracy'],
                    color=lighter_red, alpha=0.6)
    
    ax.plot(result_df_l1_hk.index + 1, result_df_l1_hk['average_accuracy'], marker='s', linestyle='-', color='#4169e0', label=r'HK')
    ax.fill_between(result_df_l1_hk.index + 1,
                    result_df_l1_hk['average_accuracy'] - result_df_l1_hk['std_accuracy'],
                    result_df_l1_hk['average_accuracy'] + result_df_l1_hk['std_accuracy'],
                    color=lighter_blue, alpha=0.6)
    ax.set_ylim(0.35, 0.7)

    # 设置标签和图例
    ax.set_xlabel(r'(b)The window of $L_2$',fontsize=14)
    ax.set_ylabel('Accuracy',fontsize=14)
    ax.legend(loc='lower right', fontsize=12)
    ax.grid(True, linestyle='--', alpha=0.7)

    # 设置字体和字号
    for label in ax.get_xticklabels() + ax.get_yticklabels():
        label.set_fontname('Times New Roman')
        label.set_fontsize(14)

    # 设置背景颜色
    fig.patch.set_facecolor('white')

    # 调整布局
    plt.tight_layout()

    # 保存图形为 PDF
    plt.savefig('D:/研究生/论文/期刊论文/小论文1/新期刊/草稿/figures/acc_l1.pdf')

    # 显示图形
    plt.show()

### 6.5.最优参数可视化

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 读取两个数据集
tbl1 = pd.read_excel('D:/研究生/论文/毕业论文/代码/网络构建/直播平台网络模型/jupyter/处理数据/dysat_0.xlsx')
tbl2 = pd.read_excel('D:/研究生/论文/毕业论文/代码/网络构建/直播平台网络模型/jupyter/处理数据/dysat_1.xlsx')

# 设置参数
metrics = ['Loss', 'Validation_accuracy', 'Test_accuracy']
num_metrics = len(metrics)

# 创建子图
fig, axes = plt.subplots(nrows=num_metrics, ncols=1, figsize=(10, 12), sharex=True)

# 绘制每个指标
for i, metric in enumerate(metrics):
    ax = axes[i]
    
    # 绘制第一个数据集
    ax.plot(tbl1.index, tbl1[metric], label=r'$L_1$', color='blue', marker='o', markersize=3, linewidth=2)
    
    # 绘制第二个数据集
    ax.plot(tbl2.index, tbl2[metric], label=r'$L_2$', color='red', marker='s', markersize=3, linewidth=2)

    # 设置标题和标签
    ax.set_title(metric, fontsize=14)
    ax.set_ylabel(metric, fontsize=12)
    ax.legend()

# 设置 X 轴标签
axes[-1].set_xlabel('窗口', fontsize=12)

# 调整布局
plt.tight_layout()

# 保存图形为 SVG
plt.savefig('D:/研究生/毕业论文/草稿/图片/opinions_dysat.svg')

# 显示图形
plt.show()


In [ ]:
# 读取 CSV 文件
csv_path = 'best_parameters_and_thresholds_l0.csv'
data = pd.read_csv(csv_path)

s1 = data['similarity_threshold']
s2 = data['state_diff_weight']
s3 = data['update_speed']
s4 = data['best_threshold']
t = np.arange(len(s1))+1

In [ ]:
# 创建共享轴图
fig, axes = plt.subplots(4, 1, figsize=(10, 8), sharex=True, constrained_layout=True,dpi=400)

# 绘制每个子图
axes[0].plot(t, s2, marker='^', label='观点权值', color='#4169e0', linewidth=2)
# axes[0].set_ylabel('Value')
axes[0].legend(loc='upper right',fontsize=10)
axes[0].grid(True, linestyle='--', alpha=0.6)

axes[1].plot(t, s1, marker='v', label='置信水平', color='#ff0000', linewidth=2)
# axes[1].set_ylabel('参数、阈值', fontsize=10.5, fontproperties=font_prop)
axes[1].legend(loc='upper right',fontsize=10)
axes[1].grid(True, linestyle='--', alpha=0.6)

axes[2].plot(t, s3, marker='<', label='更新速度', color='#00cdd0', linewidth=2)
# axes[2].set_ylabel('Value')
axes[2].legend(loc='upper right',fontsize=10)
axes[2].grid(True, linestyle='--', alpha=0.6)

axes[3].plot(t, s4, marker='o', label='行为阈值', color='#fe8b00', linewidth=2)
axes[3].set_xlabel('窗口(无单位)', fontsize=12, fontproperties=font_prop)
# axes[3].set_ylabel('Value')
axes[3].legend(loc='upper right',fontsize=10)
axes[3].grid(True, linestyle='--', alpha=0.6)

# # 设置全局标题
# fig.suptitle('Dynamic Parameters and Thresholds Over Time', fontsize=14)

# 为整个图添加居中的 Y 轴标签
fig.text(-0.02, 0.5, '参数、阈值', va='center', rotation='vertical', fontsize=12)

# 保存图形为 PDF
plt.savefig('D:/研究生/论文/毕业论文/代码/网络构建/直播平台网络模型/图片/hk_parameters_l1.svg',bbox_inches='tight')

# 显示图像
plt.show()

In [ ]:
# 创建共享轴图
fig, axes = plt.subplots(4, 1, figsize=(10, 8), sharex=True, constrained_layout=True,dpi=400)

# 绘制每个子图
axes[0].plot(t, s2, marker='^', label='Opinion Wight', color='#4169e0', linewidth=2)
# axes[0].set_ylabel('Value')
axes[0].legend(loc='upper left',fontsize=11, bbox_to_anchor=(0.08,1))
axes[0].grid(True, linestyle='--', alpha=0.6)

axes[1].plot(t, s1, marker='v', label='Confidence Level', color='#ff0000', linewidth=2)
# axes[1].set_ylabel('参数、阈值', fontsize=10.5, fontproperties=font_prop)
axes[1].legend(loc='upper left',fontsize=11, bbox_to_anchor=(0.08,1))
axes[1].grid(True, linestyle='--', alpha=0.6)

axes[2].plot(t, s3, marker='<', label='Update Speed', color='#00cdd0', linewidth=2)
# axes[2].set_ylabel('Value')
axes[2].legend(loc='upper left',fontsize=11, bbox_to_anchor=(0.08,1))
axes[2].grid(True, linestyle='--', alpha=0.6)

axes[3].plot(t, s4, marker='o', label='Behavior Threshold', color='#fe8b00', linewidth=2)
axes[3].set_xlabel('Window', fontsize=14)
# axes[3].set_ylabel('Value')
axes[3].legend(loc='upper right',fontsize=11)
axes[3].grid(True, linestyle='--', alpha=0.6)

# # 设置全局标题
# fig.suptitle('Dynamic Parameters and Thresholds Over Time', fontsize=14)

# 为整个图添加居中的 Y 轴标签
fig.text(-0.02, 0.5, 'Parameters and threshold', va='center', rotation='vertical', fontsize=14)

# 保存图形为 PDF
plt.savefig('D:/研究生/论文/期刊论文/小论文1/草稿/图片/hk_parameters_l0.eps',bbox_inches='tight')

# 显示图像
plt.show()

In [ ]:
# 读取 CSV 文件
csv_path = 'best_parameters_and_thresholds.csv'
df = pd.read_csv(csv_path)

# 提取窗口和参数
windows = df.index
similarity_threshold = df['similarity_threshold']
state_diff_weight = df['state_diff_weight']
update_speed = df['update_speed']
best_threshold = df['best_threshold']
# 创建4x1的子图网格
fig, axes = plt.subplots(nrows=4, ncols=1, figsize=(12, 20))

# 绘制每个参数的变化情况
axes[0].plot(windows, similarity_threshold, marker='o', linestyle='-', label='Similarity Threshold')
axes[0].set_xlabel('Window')
axes[0].set_ylabel('Similarity Threshold')
axes[0].legend()
axes[0].grid(True)

axes[1].plot(windows, state_diff_weight, marker='o', linestyle='-', label='State Diff Weight')
axes[1].set_xlabel('Window')
axes[1].set_ylabel('State Diff Weight')
axes[1].legend()
axes[1].grid(True)

axes[2].plot(windows, update_speed, marker='o', linestyle='-', label='Update Speed')
axes[2].set_xlabel('Window')
axes[2].set_ylabel('Update Speed')
axes[2].legend()
axes[2].grid(True)

axes[3].plot(windows, best_threshold, marker='o', linestyle='-', label='Best Threshold')
axes[3].set_xlabel('Window')
axes[3].set_ylabel('Best Threshold')
axes[3].legend()
axes[3].grid(True)

# 调整布局
plt.tight_layout()

# 显示图形
plt.show()


In [ ]:
# 保留最佳的种子数

# 读取保存最优参数的 CSV 文件
best_params_df = pd.read_csv('best_parameters_and_thresholds_l0.csv')

# 存储每个窗口下的最高 F1 分数及对应的 seed
best_results = []

# 遍历每个窗口
for start_window, changes_data in opinion_differences.items():

    # 获取当前窗口下的最优参数组合
    row = best_params_df.iloc[start_window]
    threshold = row['best_threshold']

    # 提取真实标签
    true_labels = window_labels_dict[start_window]

    # 存储每个参数组合的 F1 分数
    f1_scores = []
    seeds = []

    # 遍历每个参数组合
    for seed, opinion_changes_dict in changes_data.items():
        # 存储当前阈值下的预测标签
        binary_labels_threshold = {node_id: 0 if change > threshold else 1 for node_id, change in opinion_changes_dict.items()}

        # 检查预测标签长度与真实标签长度是否一致
        if len(true_labels) != len(binary_labels_threshold):
            continue

        # 计算 F1 分数
        _, _, f1, _ = precision_recall_fscore_support(true_labels, list(binary_labels_threshold.values()), average='weighted', zero_division=0)

        # 将 F1 分数和 seed 添加到列表中
        f1_scores.append(f1)
        seeds.append(seed)

    # 找到最高的 F1 分数及对应的 seed
    max_f1_score = max(f1_scores)
    max_f1_seed = seeds[f1_scores.index(max_f1_score)]

    # 存储每个窗口的最高 F1 分数及对应的 seed
    best_results.append({
        'window': start_window,
        'max_f1_score': max_f1_score,
        'max_f1_seed': max_f1_seed
    })

# 将结果保存到 CSV 文件
results_df = pd.DataFrame(best_results)
results_df.to_csv('best_results.csv', index=False)

print("Results have been saved to 'best_results.csv'")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

# 读取保存最优参数的 CSV 文件
best_params_df = pd.read_csv('best_parameters_and_thresholds_l1.csv')

# 存储每个窗口下的最高准确率及对应的 seed
best_results = []

# 遍历每个窗口
for start_window, changes_data in opinion_differences.items():

    # 获取当前窗口下的最优参数组合
    row = best_params_df.iloc[start_window]
    threshold = row['best_threshold']

    # 提取真实标签
    true_labels = window_labels_dict[start_window]

    # 存储每个参数组合的准确率
    acc_scores = []
    seeds = []

    # 遍历每个参数组合
    for seed, opinion_changes_dict in changes_data.items():
        # 存储当前阈值下的预测标签
        binary_labels_threshold = {node_id: 0 if change > threshold else 1 for node_id, change in opinion_changes_dict.items()}

        # 检查预测标签长度与真实标签长度是否一致
        if len(true_labels) != len(binary_labels_threshold):
            continue

        # 计算准确率
        accuracy = accuracy_score(true_labels, list(binary_labels_threshold.values()))

        # 将准确率和 seed 添加到列表中
        acc_scores.append(accuracy)
        seeds.append(seed)

    # 找到最高的准确率及对应的 seed
    max_acc_score = max(acc_scores)
    max_acc_seed = seeds[acc_scores.index(max_acc_score)]

    # 存储每个窗口的最高准确率及对应的 seed
    best_results.append({
        'window': start_window,
        'max_accuracy': max_acc_score,
        'max_accuracy_seed': max_acc_seed
    })

# 将结果保存到 CSV 文件
results_df = pd.DataFrame(best_results)
results_df.to_csv('best_results_l1.csv', index=False)

print("Results have been saved to 'best_results.csv'")


### 6.6.大网络

In [ ]:
# 加载数据
dataset_path = '../pycharm/DySAT-HK/网络数据/graph_snapshots_100_real_1.pkl'
graphs = load_snapshots(dataset_path)

window_labels_dict_big = {} 
# 外层循环遍历窗口
for start_idx in range(0, 29, 1):
    end_idx = start_idx + 5
    if end_idx > len(graphs):
        end_idx = len(graphs)

    # 选择当前窗口内的图数据
    window_graphs = graphs[start_idx:end_idx]

    active_nodes = [node for i, node in enumerate(window_graphs[3].nodes()) if
                    window_graphs[3].nodes[node]['category'] == 1]

    true_labels = [int(window_graphs[4].nodes[i]['category']) for i in active_nodes]
    
    # 将当前窗口的 true_labels 存储到字典中
    window_labels_dict_big[start_idx] = true_labels


In [ ]:
# 提取每个窗口下每个种子数对应的初始观点
# 日志文件路径
log_file = '../pycharm/DySAT-HK/logs/log_repeat-100.txt'

# 用于存储最终意见数据的嵌套字典
initial_opinions_data = {}

# 定义正则表达式
opinions_pattern = r'Window: (\d+)-(\d+), Seed: (\d+), Initial Opinions: \{([^}]+)\}'

# 读取日志文件并处理每一行
with open(log_file, 'r') as file:
    lines = file.readlines()
    for line in lines:
        match = re.search(opinions_pattern, line)
        if match:
            start_window = int(match.group(1))
            end_window = int(match.group(2))
            seed = int(match.group(3))
            opinions_str = match.group(4)
            
            # 解析 Opinions 字符串
            opinions = {}
            for item in opinions_str.split(','):
                node, opinion = item.split(':')
                opinions[int(node.strip())] = float(opinion.strip())
            
            # 将数据存储到嵌套字典中
            if start_window not in initial_opinions_data:
                initial_opinions_data[start_window] = {}
            initial_opinions_data[start_window][seed] = opinions

In [ ]:
# 提取每个窗口下每个种子数对应的最终观点

# 日志文件路径
log_file = '../pycharm/DySAT-HK/logs/log_repeat-100.txt'

# 用于存储最终意见数据的嵌套字典
final_opinions_data = {}

# 定义正则表达式
opinions_pattern = r'Window: (\d+)-(\d+), Seed: (\d+), Final Opinions: \{([^}]+)\}'

# 读取日志文件并处理每一行
with open(log_file, 'r') as file:
    lines = file.readlines()
    for line in lines:
        match = re.search(opinions_pattern, line)
        if match:
            start_window = int(match.group(1))
            end_window = int(match.group(2))
            seed = int(match.group(3))
            opinions_str = match.group(4)
            
            # 解析 Opinions 字符串
            opinions = {}
            for item in opinions_str.split(','):
                node, opinion = item.split(':')
                opinions[int(node.strip())] = float(opinion.strip())
            
            # 将数据存储到嵌套字典中
            if start_window not in final_opinions_data:
                final_opinions_data[start_window] = {}
            final_opinions_data[start_window][seed] = opinions

In [ ]:
# 存储每个窗口下各个节点的观点差异
opinion_differences = {}

# 遍历每个窗口
for window in initial_opinions_data:
    if window not in final_opinions_data:
        continue

    # 遍历每组参数组合
    for seed in final_opinions_data[window]:
        initial_opinions = initial_opinions_data[window][seed]
        final_opinions = final_opinions_data[window][seed]

        # 计算观点差异的绝对值
        differences = {}
        for node in initial_opinions:
            if node in final_opinions:
                difference = abs(final_opinions[node] - initial_opinions[node])
                differences[node] = difference

        # 将数据存储到嵌套字典中
        if window not in opinion_differences:
            opinion_differences[window] = {}
        opinion_differences[window][seed] = differences

In [ ]:
# 计算每个窗口100次实验的平均评估指标值

# 读取保存最优参数的 CSV 文件
best_params_df = pd.read_csv('best_parameters_and_thresholds.csv')

# 存储每个窗口下的评估结果
best_results = {}

# 遍历每个窗口
for start_window, changes_data in opinion_differences.items():

    # 获取当前窗口下的最优参数组合
    row = best_params_df.iloc[start_window]
    threshold = row['best_threshold']

    # 提取真实标签
    true_labels = window_labels_dict_big[start_window]

    # 存储每个参数组合的最佳 F1 分数
    f1_scores = []
    precisions = []
    recalls = []

    # 遍历每个参数组合
    for seed, opinion_changes_dict in changes_data.items():
        # 存储当前阈值下的预测标签
        binary_labels_threshold = {node_id: 0 if change > threshold else 1 for node_id, change in opinion_changes_dict.items()}

        # 检查预测标签长度与真实标签长度是否一致
        if len(true_labels) != len(binary_labels_threshold):
            continue

        # 计算 F1 分数、recall 和 precision
        precision, recall, f1, _ = precision_recall_fscore_support(true_labels, list(binary_labels_threshold.values()), average='weighted', zero_division=0)

        # 将分数添加到列表中
        f1_scores.append(f1)
        precisions.append(precision)
        recalls.append(recall)

    # 计算平均值
    avg_f1_score = np.mean(f1_scores)
    avg_precision = np.mean(precisions)
    avg_recall = np.mean(recalls)

    # 计算标准差
    std_f1_score = np.std(f1_scores)
    std_precision = np.std(precisions)
    std_recall = np.std(recalls)

    # 存储每个窗口的平均评估结果及其标准差
    best_results[start_window] = {
        'average_f1_score': avg_f1_score,
        'std_f1_score': std_f1_score,
        'average_precision': avg_precision,
        'std_precision': std_precision,
        'average_recall': avg_recall,
        'std_recall': std_recall
    }

# 将结果保存为 CSV 文件
result_df = pd.DataFrame(best_results).T  # 转置 DataFrame 使窗口编号为行索引
csv_path = 'window_evaluation_results_big_true.csv'
result_df.to_csv(csv_path, index_label='window')

# 打印路径以供参考
print(f"CSV文件已保存至: {csv_path}")


In [ ]:
# 读取保存的评估结果 CSV 文件
result_df = pd.read_csv('window_evaluation_results.csv', index_col='window')
result_df_big = pd.read_csv('window_evaluation_results_big.csv', index_col='window')

lighter_blue = '#d0e1f9'   # 浅浅蓝色
lighter_red = '#ffcccc'    # 浅浅红色
lighter_cyan = '#ccf2f2'   # 浅浅青色

# 创建子图
fig, ax = plt.subplots(figsize=(8, 6))

# 绘制 F1 分数及其标准差阴影
ax.plot(result_df.index + 1, result_df['average_f1_score'], marker='o', linestyle='-', color='#00cdd0', label=r'$N_0=50$')
ax.fill_between(result_df.index + 1,
                result_df['average_f1_score'] - result_df['std_f1_score'],
                result_df['average_f1_score'] + result_df['std_f1_score'],
                color=lighter_cyan)

# 绘制 F1 分数及其标准差阴影
ax.plot(result_df_big.index + 1, result_df_big['average_f1_score'], marker='o', linestyle='-', color='#4169e0', label=r'$N_0=100$')
ax.fill_between(result_df_big.index + 1,
                result_df_big['average_f1_score'] - result_df_big['std_f1_score'],
                result_df_big['average_f1_score'] + result_df_big['std_f1_score'],
                color=lighter_blue)

# 设置标签和图例
ax.set_xlabel('Window', fontsize=16)
ax.set_ylabel('$F_1$', fontsize=16)
ax.legend(loc='best', fontsize=16)
ax.grid(True, linestyle='--', alpha=0.7)

# 设置字体和字号
for label in ax.get_xticklabels() + ax.get_yticklabels():
    label.set_fontname('Times New Roman')
    label.set_fontsize(16)

# 设置背景颜色
fig.patch.set_facecolor('white')

# 调整布局
plt.tight_layout()

# 保存图形为 PDF
plt.savefig('D:/lunwen/social_network2023.7.17/caogao/Applied Soft Computing/figures/f1_big.eps', dpi=300)

# 显示图形
plt.show()


### 6.7.验证网络

In [ ]:
# 定义日志文件路径
log_file = '../pycharm/DySAT-HK/logs/log_danci_val.txt'

# 用于存储最终意见数据的嵌套字典
initial_opinions_data = {}

# 定义正则表达式
opinions_pattern = r'Window: (\d+)-(\d+), Initial Opinions: \{([^}]+)\}'

# 读取日志文件并处理每一行
with open(log_file, 'r') as file:
    lines = file.readlines()
    for line in lines:
        match = re.search(opinions_pattern, line)
        if match:
            start_window = int(match.group(1))
            end_window = int(match.group(2))
            opinions_str = match.group(3)
            
            # 解析 Opinions 字符串
            opinions = {}
            for item in opinions_str.split(','):
                node, opinion = item.split(':')
                opinions[int(node.strip())] = float(opinion.strip())
            
            # 将数据存储到嵌套字典中
            if start_window not in initial_opinions_data:
                initial_opinions_data[start_window] = {}
            initial_opinions_data[start_window] = opinions

In [ ]:
# 定义日志文件路径
log_file = '../pycharm/DySAT-HK/logs/log_danci_val.txt'

# 用于存储最终意见数据的嵌套字典
final_opinions_data = {}

# 定义正则表达式
opinions_pattern = r'Window: (\d+)-(\d+), Parameters: \(([^,]+), ([^,]+), ([^)]+)\), Final Opinions: \{([^}]+)\}'

# 读取日志文件并处理每一行
with open(log_file, 'r') as file:
    lines = file.readlines()
    for line in lines:
        match = re.search(opinions_pattern, line)
        if match:
            start_window = int(match.group(1))
            end_window = int(match.group(2))
            param1 = float(match.group(3))
            param2 = float(match.group(4))
            param3 = float(match.group(5))
            opinions_str = match.group(6)
            
            # 解析 Opinions 字符串
            opinions = {}
            for item in opinions_str.split(','):
                node, opinion = item.split(':')
                opinions[int(node.strip())] = float(opinion.strip())
            
            # 将数据存储到嵌套字典中
            if start_window not in final_opinions_data:
                final_opinions_data[start_window] = {}
            final_opinions_data[start_window][(param1, param2, param3)] = opinions

In [ ]:
opinion_differences = {}

# 遍历每个窗口
for window in initial_opinions_data:
    initial_opinions = initial_opinions_data[window]
    if window not in final_opinions_data:
        continue

    # 遍历每组参数组合
    for params in final_opinions_data[window]:
        final_opinions = final_opinions_data[window][params]

        # 计算观点差异的绝对值
        differences = {}
        for node in initial_opinions:
            if node in final_opinions:
                difference = abs(final_opinions[node] - initial_opinions[node])
                differences[node] = difference

        # 将数据存储到嵌套字典中
        if window not in opinion_differences:
            opinion_differences[window] = {}
        opinion_differences[window][params] = differences


In [ ]:
# 加载数据
data = np.load('../pycharm/DySAT_pytorch-main/data/DBLPE/DBLPE.npz')
adjs_matrix = data['adjs']
labels = data['labels'][:, :, 0]
# 创建一个空列表用于存储图形对象
graphs = []
# 循环迭代每个时间步的邻接矩阵
for time_step in range(adjs_matrix.shape[0]):
    # 获取特定时间步的邻接矩阵
    adj_matrix_at_time_step = adjs_matrix[time_step, :, :]
    # 获取当前时间步的标签矩阵
    current_labels_matrix = labels[:, time_step]
    # 将邻接矩阵转换为图形对象
    G = nx.from_numpy_array(adj_matrix_at_time_step)
    # 将标签矩阵的每行的第一个值作为节点标签
    for i, node in enumerate(G.nodes()):
        G.nodes[node]['label'] = current_labels_matrix[i]
    # 将图形对象添加到列表中
    graphs.append(G)

window_labels_dict_val = {} 
# 外层循环遍历窗口
for start_idx in range(0, 3, 1):
    end_idx = start_idx + 5
    if end_idx > len(graphs):
        end_idx = len(graphs)

    # 选择当前窗口内的图数据
    window_graphs = graphs[start_idx:end_idx]

    active_nodes = [node for i, node in enumerate(window_graphs[3].nodes()) if
                    window_graphs[3].nodes[node]['label'] == 1]

    true_labels = [int(window_graphs[4].nodes[i]['label']) for i in active_nodes]
    
    # 将当前窗口的 true_labels 存储到字典中
    window_labels_dict_val[start_idx] = true_labels


In [ ]:
# 定义阈值的范围
thresholds = np.arange(0.1, 0.55, step=0.1)

# 存储每个窗口下最佳阈值和对应的评估结果
best_thresholds_and_results = {}

# 遍历每个窗口
for start_window, changes_data in opinion_differences.items():
    
    # 提取真实标签
    true_labels = window_labels_dict_val[start_window]

    # 存储每个参数组合的最佳阈值和对应的最佳 F1 分数
    best_thresholds_and_f1 = []

    # 遍历每个参数组合
    for params, opinion_changes_dict in changes_data.items():
        # 存储当前参数组合的最佳阈值和对应的最佳 F1 分数
        best_f1_score = 0
        best_precision = 0
        best_recall = 0
        best_threshold = None

        # 遍历每个阈值
        for threshold in thresholds:
            # 存储当前阈值下的预测标签
            binary_labels_threshold = {node_id: 0 if change > threshold else 1 for node_id, change in opinion_changes_dict.items()}

            # 检查预测标签长度与真实标签长度是否一致
            if len(true_labels) != len(binary_labels_threshold):
                continue

            # 计算 F1 分数、recall 和 precision
            precision, recall, f1, _ = precision_recall_fscore_support(true_labels, list(binary_labels_threshold.values()), average='weighted', zero_division=0)

            # 更新最佳 F1 分数和对应的阈值
            if f1 > best_f1_score:
                best_f1_score = f1
                best_precision = precision
                best_recall = recall
                best_threshold = threshold

        # 在每个参数组合下，存储最佳阈值和对应的最佳 F1 分数
        best_thresholds_and_f1.append({
            'parameters': params,
            'best_threshold': best_threshold,
            'best_f1_score': best_f1_score,
            'precision': best_precision,
            'recall': best_recall
        })

    # 在每个窗口下，找到具有最佳 F1 分数的参数组合和阈值
    best_result = max(best_thresholds_and_f1, key=lambda x: x['best_f1_score'])

    # 存储最佳阈值和评估结果
    best_thresholds_and_results[start_window] = {
        'best_parameters': best_result['parameters'],
        'best_threshold': best_result['best_threshold'],
        'precision': best_result['precision'],
        'recall': best_result['recall'],
        'f1_score': best_result['best_f1_score']
    }


In [ ]:
best_thresholds_and_results

In [ ]:
# 创建一个空的DataFrame
result = []
# 遍历每个窗口
for window_key, data_list in best_thresholds_and_results.items():

    best_parameters = data_list['best_parameters']
    best_threshold = data_list['best_threshold']
    result.append({
        'similarity_threshold': best_parameters[0],
        'state_diff_weight': best_parameters[1],
        'update_speed': best_parameters[2],
        'best_threshold': best_threshold,
    })

# 保存为CSV文件
df = pd.DataFrame(result)
csv_path = 'best_parameters_and_thresholds_val.csv'
df.to_csv(csv_path, index=False)

# 打印路径以供参考
print(f"CSV文件已保存至: {csv_path}")


In [ ]:
# 提取每个窗口下每个种子数对应的初始观点
# 日志文件路径
log_file = '../pycharm/DySAT-HK/logs/log_repeat-100_val.txt'

# 用于存储最终意见数据的嵌套字典
initial_opinions_data = {}

# 定义正则表达式
opinions_pattern = r'Window: (\d+)-(\d+), Seed: (\d+), Initial Opinions: \{([^}]+)\}'

# 读取日志文件并处理每一行
with open(log_file, 'r') as file:
    lines = file.readlines()
    for line in lines:
        match = re.search(opinions_pattern, line)
        if match:
            start_window = int(match.group(1))
            end_window = int(match.group(2))
            seed = int(match.group(3))
            opinions_str = match.group(4)
            
            # 解析 Opinions 字符串
            opinions = {}
            for item in opinions_str.split(','):
                node, opinion = item.split(':')
                opinions[int(node.strip())] = float(opinion.strip())
            
            # 将数据存储到嵌套字典中
            if start_window not in initial_opinions_data:
                initial_opinions_data[start_window] = {}
            initial_opinions_data[start_window][seed] = opinions

In [ ]:
# 提取每个窗口下每个种子数对应的最终观点

# 日志文件路径
log_file = '../pycharm/DySAT-HK/logs/log_repeat-100_val.txt'

# 用于存储最终意见数据的嵌套字典
final_opinions_data = {}

# 定义正则表达式
opinions_pattern = r'Window: (\d+)-(\d+), Seed: (\d+), Final Opinions: \{([^}]+)\}'

# 读取日志文件并处理每一行
with open(log_file, 'r') as file:
    lines = file.readlines()
    for line in lines:
        match = re.search(opinions_pattern, line)
        if match:
            start_window = int(match.group(1))
            end_window = int(match.group(2))
            seed = int(match.group(3))
            opinions_str = match.group(4)
            
            # 解析 Opinions 字符串
            opinions = {}
            for item in opinions_str.split(','):
                node, opinion = item.split(':')
                opinions[int(node.strip())] = float(opinion.strip())
            
            # 将数据存储到嵌套字典中
            if start_window not in final_opinions_data:
                final_opinions_data[start_window] = {}
            final_opinions_data[start_window][seed] = opinions

In [ ]:
# 存储每个窗口下各个节点的观点差异
opinion_differences = {}

# 遍历每个窗口
for window in initial_opinions_data:
    if window not in final_opinions_data:
        continue

    # 遍历每组参数组合
    for seed in final_opinions_data[window]:
        initial_opinions = initial_opinions_data[window][seed]
        final_opinions = final_opinions_data[window][seed]

        # 计算观点差异的绝对值
        differences = {}
        for node in initial_opinions:
            if node in final_opinions:
                difference = abs(final_opinions[node] - initial_opinions[node])
                differences[node] = difference

        # 将数据存储到嵌套字典中
        if window not in opinion_differences:
            opinion_differences[window] = {}
        opinion_differences[window][seed] = differences

In [ ]:
# 计算每个窗口100次实验的平均评估指标值

import pandas as pd
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

# 读取保存最优参数的 CSV 文件
best_params_df = pd.read_csv('best_parameters_and_thresholds_val.csv')

# 存储每个窗口下的评估结果
best_results = {}

# 遍历每个窗口
for start_window, changes_data in opinion_differences.items():

    # 获取当前窗口下的最优参数组合
    row = best_params_df.iloc[start_window]
    threshold = row['best_threshold']

    # 提取真实标签
    true_labels = window_labels_dict_val[start_window]

    # 存储每个参数组合的最佳 F1 分数
    f1_scores = []
    precisions = []
    recalls = []

    # 遍历每个参数组合
    for seed, opinion_changes_dict in changes_data.items():
        # 存储当前阈值下的预测标签
        binary_labels_threshold = {node_id: 0 if change > threshold else 1 for node_id, change in opinion_changes_dict.items()}

        # 检查预测标签长度与真实标签长度是否一致
        if len(true_labels) != len(binary_labels_threshold):
            continue

        # 计算 F1 分数、recall 和 precision
        precision, recall, f1, _ = precision_recall_fscore_support(true_labels, list(binary_labels_threshold.values()), average='weighted', zero_division=0)

        # 将分数添加到列表中
        f1_scores.append(f1)
        precisions.append(precision)
        recalls.append(recall)

    # 计算平均值
    avg_f1_score = np.mean(f1_scores)
    avg_precision = np.mean(precisions)
    avg_recall = np.mean(recalls)

    # 计算标准差
    std_f1_score = np.std(f1_scores)
    std_precision = np.std(precisions)
    std_recall = np.std(recalls)

    # 存储每个窗口的平均评估结果及其标准差
    best_results[start_window] = {
        'average_f1_score': avg_f1_score,
        'std_f1_score': std_f1_score,
        'average_precision': avg_precision,
        'std_precision': std_precision,
        'average_recall': avg_recall,
        'std_recall': std_recall
    }

# 将结果保存为 CSV 文件
result_df = pd.DataFrame(best_results).T  # 转置 DataFrame 使窗口编号为行索引
csv_path = 'window_evaluation_results_val.csv'
result_df.to_csv(csv_path, index_label='window')

# 打印路径以供参考
print(f"CSV文件已保存至: {csv_path}")


### 6.8.同一窗口下不同参数的观点演化

In [ ]:
# 定义日志文件路径
log_file = '../python/Multi_DySAT-HK/logs/log_win15_l0.txt'

# 用于存储最终意见数据的嵌套字典
initial_opinions_data = {}

# 定义正则表达式
opinions_pattern = r'Parameters: \(([^,]+), ([^,]+), ([^)]+)\), Iteration: (\d+), Initial Opinions: \{([^}]+)\}'

# 读取日志文件并处理每一行
with open(log_file, 'r') as file:
    lines = file.readlines()
    for line in lines:
        match = re.search(opinions_pattern, line)
        if match:
            param1 = float(match.group(2))
            param2 = float(match.group(1))
            param3 = float(match.group(3))
            iteration = int(match.group(4))
            opinions_str = match.group(5)
            
            # 解析 Opinions 字符串
            opinions = {}
            for item in opinions_str.split(','):
                node, opinion = item.split(':')
                opinions[int(node.strip())] = float(opinion.strip())
            
            # 将数据存储到嵌套字典中
            if (param1,param2,param3) not in initial_opinions_data:
                initial_opinions_data[(param1,param2,param3)] = {}
            if iteration not in initial_opinions_data[(param1,param2,param3)]:
                initial_opinions_data[(param1,param2,param3)][iteration] = opinions


In [ ]:
initial_opinions_data

In [ ]:
# 定义日志文件路径
log_file = '../python/Multi_DySAT-HK/logs/log_win15_l0.txt'

# 用于存储最终意见数据的嵌套字典
final_opinions_data = {}

# 定义正则表达式
opinions_pattern = r'Parameters: \(([^,]+), ([^,]+), ([^)]+)\), Iteration: (\d+), Opinions: \{([^}]+)\}'

# 读取日志文件并处理每一行
with open(log_file, 'r') as file:
    lines = file.readlines()
    for line in lines:
        match = re.search(opinions_pattern, line)
        if match:
            param1 = float(match.group(2))
            param2 = float(match.group(1))
            param3 = float(match.group(3))
            iteration = int(match.group(4))-1
            opinions_str = match.group(5)
            
            # 解析 Opinions 字符串
            opinions = {}
            for item in opinions_str.split(','):
                node, opinion = item.split(':')
                opinions[int(node.strip())] = float(opinion.strip())
            
            # 将数据存储到嵌套字典中
            if (param1,param2,param3) not in final_opinions_data:
                final_opinions_data[(param1,param2,param3)] = {}
            if iteration not in final_opinions_data[(param1,param2,param3)]:
                final_opinions_data[(param1,param2,param3)][iteration] = opinions

In [ ]:
# 合并后的字典
merged_opinions_data = {}

# 遍历 initial_opinions_data 并合并
for params, initial_data in initial_opinions_data.items():
    # 初始化 merged_opinions_data 的键值对
    merged_opinions_data[params] = {}

    # 添加初始观点数据
    merged_opinions_data[params].update(initial_data)

    # 如果 final_opinions_data 中有相同的参数组合，则添加最终观点数据
    if params in final_opinions_data:
        merged_opinions_data[params].update(final_opinions_data[params])

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

# 要筛选的特定参数组合
selected_params = [(0.4, 0.2, 0.1), (0.4, 0.2, 0.2), (0.4, 0.2, 0.3), (0.4, 0.2, 0.4), (0.4, 0.2, 0.5), (0.4, 0.2, 0.6), (0.4, 0.2, 0.7), (0.4, 0.2, 0.8), (0.4, 0.2, 0.9)]

# [(0.7, 0.3, 0.1), (0.7, 0.3, 0.2), (0.7, 0.3, 0.3), (0.7, 0.3, 0.4), (0.7, 0.3, 0.5), (0.7, 0.3, 0.6), (0.7, 0.3, 0.7), (0.7, 0.3, 0.8), (0.7, 0.3, 0.9)]

# [(0.7, 0.1, 0.3), (0.7,0.2, 0.3), (0.7, 0.3, 0.3), (0.7, 0.4, 0.3), (0.7, 0.5, 0.3), (0.7, 0.6, 0.3), (0.7, 0.7, 0.3), (0.7,0.8, 0.3), (0.7, 0.9, 0.3)]

# [(0.1, 0.3, 0.3), (0.2,0.3, 0.3), (0.3, 0.3, 0.3), (0.4, 0.3, 0.3), (0.5, 0.3, 0.3), (0.6, 0.3, 0.3), (0.7, 0.3, 0.3), (0.8,0.3, 0.3), (0.9, 0.3, 0.3)]

# [(0.4, 0.2, 0.1), (0.4, 0.2, 0.2), (0.4, 0.2, 0.3), (0.4, 0.2, 0.4), (0.4, 0.2, 0.5), (0.4, 0.2, 0.6), (0.4, 0.2, 0.7), (0.4, 0.2, 0.8), (0.4, 0.2, 0.9)]

# [(0.4, 0.1, 0.9), (0.4, 0.2, 0.9), (0.4, 0.3, 0.9), (0.4, 0.4, 0.9), (0.4, 0.5, 0.9), (0.4, 0.6, 0.9), (0.4, 0.7, 0.9), (0.4, 0.8, 0.9), (0.4, 0.9, 0.9)]

# [(0.1, 0.2, 0.9), (0.2,0.2, 0.9), (0.3, 0.2, 0.9), (0.4,0.2, 0.9), (0.5, 0.2, 0.9), (0.6, 0.2, 0.9), (0.7, 0.2, 0.9), (0.8,0.2, 0.9), (0.9, 0.2, 0.9)]

# 提取特定参数组合的数据
selected_data = {params: merged_opinions_data[params] for params in selected_params}

# 提取真实标签
true_labels = window_labels_dict[14]

# 进行循环之前，创建一个空的图例列表
legend_elements = []

# 设置子图的规格为3x3
num_params = len(selected_params)
num_rows = 3
num_cols = 3
fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(12, 6))
axes = axes.flatten()  # 将子图的二维数组展平成一维数组，方便索引

# 遍历每个参数组合，并在对应的子图上绘制
for idx, (params, data) in enumerate(selected_data.items()):
    ax = axes[idx]

    iterations = []
    opinions = []

    # 各个迭代次数对应的观点值
    for iteration, opinions_data in data.items():
        iterations.append(iteration)
        # 存储观点数据
        opinions.append(opinions_data)

    # 绘制观点演化图
    for node, values in opinions[0].items():
        node_opinions = [opinions_data[node] for opinions_data in opinions]

        # 获取节点的位置索引
        node_index = list(opinions[0].keys()).index(node)

        # 根据行为值选择颜色
        color = '#6390eb' if true_labels[node_index] == 1 else '#fe6a6a'  # 根据位置索引选择颜色

        # 绘制观点演化图，并使用不同颜色
        ax.plot(iterations, node_opinions, color=color, linestyle='-')

        # 添加图例元素
        label = '保持在线的节点' if true_labels[node_index] == 1 else '在线转化为离线的节点'
        if label not in [leg.get_label() for leg in legend_elements]:
            legend_elements.append(Line2D([0], [0], color=color, label=label))
    
    # 在子图底部添加参数信息
    ax.text(0.5, -0.3, f'({idx+1})  (δ, ε, λ) = ({params[0]}, {params[1]}, {params[2]})',
            ha='center', va='center', fontsize=12, fontproperties=font_prop, transform=ax.transAxes)


    # 只在第一列添加 y 轴标签
    if idx % num_cols == 0:
        ax.set_ylabel('观点', fontsize=12, fontproperties=font_prop)

    # 只在最后一排添加 x 轴标签
    if idx // num_cols == num_rows - 1:
        ax.set_xlabel('时间', fontsize=12, fontproperties=font_prop,labelpad=20)

    # 设置坐标轴刻度标签字体大小
    ax.tick_params(axis='both', which='major', labelsize=10)

    # 使用整数坐标
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))

# 添加图例，设置为一行放两个元素并放在图的正下方
fig.legend(handles=legend_elements, loc='lower center', fontsize=12, bbox_to_anchor=(0.5, -0.05), ncol=2)


# 隐藏剩余的子图（如果有的话）
if num_params < num_rows * num_cols:
    for ax in axes[num_params:]:
        ax.axis('off')

# 调整布局
plt.tight_layout()

# 保存图形为 PDF
plt.savefig('D:/研究生/论文/毕业论文/草稿/图片/opinions_3_l0.svg', bbox_inches='tight')

plt.show()


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

# 要筛选的特定参数组合
# selected_params = [(0.1, 0.4, 0.9), (0.2,0.4, 0.9), (0.3, 0.4, 0.9), (0.4,0.4, 0.9), (0.5, 0.4, 0.9), (0.6, 0.4, 0.9), (0.7, 0.4, 0.9), (0.8,0.4, 0.9), (0.9, 0.4, 0.9)]
# selected_params = [(0.1, 0.7, 0.3), (0.2,0.7, 0.3), (0.3, 0.7, 0.3), (0.4, 0.7, 0.3), (0.5, 0.7, 0.3), (0.6, 0.7, 0.3), (0.7, 0.7, 0.3), (0.8,0.7, 0.3), (0.9, 0.7, 0.3)]

# [(0.7, 0.3, 0.1), (0.7, 0.3, 0.2), (0.7, 0.3, 0.3), (0.7, 0.3, 0.4), (0.7, 0.3, 0.5), (0.7, 0.3, 0.6), (0.7, 0.3, 0.7), (0.7, 0.3, 0.8), (0.7, 0.3, 0.9)]

# [(0.7, 0.1, 0.3), (0.7,0.2, 0.3), (0.7, 0.3, 0.3), (0.7, 0.4, 0.3), (0.7, 0.5, 0.3), (0.7, 0.6, 0.3), (0.7, 0.7, 0.3), (0.7,0.8, 0.3), (0.7, 0.9, 0.3)]

# [(0.1, 0.3, 0.3), (0.2,0.3, 0.3), (0.3, 0.3, 0.3), (0.4, 0.3, 0.3), (0.5, 0.3, 0.3), (0.6, 0.3, 0.3), (0.7, 0.3, 0.3), (0.8,0.3, 0.3), (0.9, 0.3, 0.3)]

selected_params = [(0.4, 0.2, 0.1), (0.4, 0.2, 0.2), (0.4, 0.2, 0.3), (0.4, 0.2, 0.4), (0.4, 0.2, 0.5), (0.4, 0.2, 0.6), (0.4, 0.2, 0.7), (0.4, 0.2, 0.8), (0.4, 0.2, 0.9)]

# [(0.4, 0.1, 0.9), (0.4, 0.2, 0.9), (0.4, 0.3, 0.9), (0.4, 0.4, 0.9), (0.4, 0.5, 0.9), (0.4, 0.6, 0.9), (0.4, 0.7, 0.9), (0.4, 0.8, 0.9), (0.4, 0.9, 0.9)]

# [(0.1, 0.2, 0.9), (0.2,0.2, 0.9), (0.3, 0.2, 0.9), (0.4,0.2, 0.9), (0.5, 0.2, 0.9), (0.6, 0.2, 0.9), (0.7, 0.2, 0.9), (0.8,0.2, 0.9), (0.9, 0.2, 0.9)]


# 提取特定参数组合的数据
selected_data = {params: merged_opinions_data[params] for params in selected_params}

# 提取真实标签
true_labels = window_labels_dict[14]

# 进行循环之前，创建一个空的图例列表
legend_elements = []

# 设置子图的规格为3x3
num_params = len(selected_params)
num_rows = 3
num_cols = 3
fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(12, 6))
axes = axes.flatten()  # 将子图的二维数组展平成一维数组，方便索引

# 遍历每个参数组合，并在对应的子图上绘制
for idx, (params, data) in enumerate(selected_data.items()):
    ax = axes[idx]

    iterations = []
    opinions = []

    # 各个迭代次数对应的观点值
    for iteration, opinions_data in data.items():
        iterations.append(iteration)
        # 存储观点数据
        opinions.append(opinions_data)

    # 绘制观点演化图
    for node, values in opinions[0].items():
        node_opinions = [opinions_data[node] for opinions_data in opinions]

        # 获取节点的位置索引
        node_index = list(opinions[0].keys()).index(node)

        # 根据行为值选择颜色
        color = '#6390eb' if true_labels[node_index] == 1 else '#fe6a6a'  # 根据位置索引选择颜色

        # 绘制观点演化图，并使用不同颜色
        ax.plot(iterations, node_opinions, color=color, linestyle='-')

        # 添加图例元素
        label = 'Node that Stays Online' if true_labels[node_index] == 1 else 'Node that Switches from Online to Offline'
        if label not in [leg.get_label() for leg in legend_elements]:
            legend_elements.append(Line2D([0], [0], color=color, label=label))
    
    # 在子图底部添加参数信息
    ax.text(0.5, -0.3, f'({idx+1})  (δ, λ, ε) = ({params[0]}, {params[1]}, {params[2]})',
            ha='center', va='center', fontsize=12, transform=ax.transAxes)


    # 只在第一列添加 y 轴标签
    if idx % num_cols == 0:
        ax.set_ylabel('Opinion', fontsize=12)

    # 只在最后一排添加 x 轴标签
    if idx // num_cols == num_rows - 1:
        ax.set_xlabel('Time', fontsize=12,labelpad=20)

    # 设置坐标轴刻度标签字体大小
    ax.tick_params(axis='both', which='major', labelsize=10)

    # 使用整数坐标
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))

# 添加图例，设置为一行放两个元素并放在图的正下方
fig.legend(handles=legend_elements, loc='lower center', fontsize=12, bbox_to_anchor=(0.5, -0.05), ncol=2)


# 隐藏剩余的子图（如果有的话）
if num_params < num_rows * num_cols:
    for ax in axes[num_params:]:
        ax.axis('off')

# 调整布局
plt.tight_layout()

# 保存图形为 PDF
plt.savefig('D:/研究生/论文/期刊论文/小论文1/草稿/图片/opinions_3_l0.eps', bbox_inches='tight')

plt.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

# 要筛选的特定参数组合
selected_params = [(0.2, 0.1, 0.9), (0.2, 0.2, 0.9), (0.2, 0.3, 0.9), (0.2, 0.4, 0.9), (0.2, 0.5, 0.9), (0.2, 0.6, 0.9), (0.2, 0.7, 0.9), (0.2, 0.8, 0.9), (0.2, 0.9, 0.9)]
# selected_params = [(0.3, 0.1, 0.3), (0.3, 0.2, 0.3), (0.3, 0.3, 0.3), (0.3, 0.4, 0.3), (0.3, 0.5, 0.3), (0.3, 0.6, 0.3), (0.3, 0.7, 0.3), (0.3, 0.8, 0.3), (0.3, 0.9, 0.3)]

# 提取特定参数组合的数据
selected_data = {params: merged_opinions_data[params] for params in selected_params}

# 提取真实标签
true_labels = window_labels_dict[14]
# 进行循环之前，创建一个空的图例列表
legend_elements = []

# 设置子图的规格为3x3
num_params = len(selected_params)
num_rows = 3
num_cols = 3
fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(15, 6))
axes = axes.flatten()  # 将子图的二维数组展平成一维数组，方便索引

# 遍历每个参数组合，并在对应的子图上绘制
for idx, (params, data) in enumerate(selected_data.items()):
    ax = axes[idx]

    iterations = []
    opinions = []

    # 各个迭代次数对应的观点值
    for iteration, opinions_data in data.items():
        iterations.append(iteration)
        # 存储观点数据
        opinions.append(opinions_data)

    # 绘制观点演化图
    for node, values in opinions[0].items():
        node_opinions = [opinions_data[node] for opinions_data in opinions]

        # 获取节点的位置索引
        node_index = list(opinions[0].keys()).index(node)

        # 根据行为值选择颜色
        color = '#6390eb' if true_labels[node_index] == 1 else '#fe6a6a'  # 根据位置索引选择颜色

        # 绘制观点演化图，并使用不同颜色
        ax.plot(iterations, node_opinions, color=color, linestyle='-')

        # 添加图例元素
        label = '保持在线的节点' if true_labels[node_index] == 1 else '在线转化为离线的节点'
        if label not in [leg.get_label() for leg in legend_elements]:
            legend_elements.append(Line2D([0], [0], color=color, label=label))
    
    # 在子图底部添加参数信息
    ax.text(0.5, -0.3, f'({idx+1})  (δ, λ, ε) = ({params[0]}, {params[1]}, {params[2]})',
            ha='center', va='center', fontsize=12, fontproperties=font_prop, transform=ax.transAxes)


    # 只在第一列添加 y 轴标签
    if idx % num_cols == 0:
        ax.set_ylabel('观点', fontsize=12, fontproperties=font_prop)

    # 只在最后一排添加 x 轴标签
    if idx // num_cols == num_rows - 1:
        ax.set_xlabel('时间', fontsize=12, fontproperties=font_prop,labelpad=20)

    # 设置坐标轴刻度标签字体大小
    ax.tick_params(axis='both', which='major', labelsize=10)

    # 使用整数坐标
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))

# 添加图例，设置为一行放两个元素并放在图的正下方
fig.legend(handles=legend_elements, loc='lower center', fontsize=12, bbox_to_anchor=(0.5, -0.05), ncol=2)

# 隐藏剩余的子图（如果有的话）
if num_params < num_rows * num_cols:
    for ax in axes[num_params:]:
        ax.axis('off')

# 调整布局
plt.tight_layout()

# 保存图形为 PDF
plt.savefig('D:/研究生/毕业论文/草稿/图片/opinions_2_l0.svg', bbox_inches='tight')

plt.show()


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

# 要筛选的特定参数组合
# selected_params = [(0.2, 0.1, 0.9), (0.2, 0.2, 0.9), (0.2, 0.3, 0.9), (0.2, 0.4, 0.9), (0.2, 0.5, 0.9), (0.2, 0.6, 0.9), (0.2, 0.7, 0.9), (0.2, 0.8, 0.9), (0.2, 0.9, 0.9)]
selected_params = [(0.3, 0.1, 0.3), (0.3, 0.2, 0.3), (0.3, 0.3, 0.3), (0.3, 0.4, 0.3), (0.3, 0.5, 0.3), (0.3, 0.6, 0.3), (0.3, 0.7, 0.3), (0.3, 0.8, 0.3), (0.3, 0.9, 0.3)]

# 提取特定参数组合的数据
selected_data = {params: merged_opinions_data[params] for params in selected_params}

# 提取真实标签
true_labels = window_labels_dict[14]
# 进行循环之前，创建一个空的图例列表
legend_elements = []

# 设置子图的规格为3x3
num_params = len(selected_params)
num_rows = 3
num_cols = 3
fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(15, 6))
axes = axes.flatten()  # 将子图的二维数组展平成一维数组，方便索引

# 遍历每个参数组合，并在对应的子图上绘制
for idx, (params, data) in enumerate(selected_data.items()):
    ax = axes[idx]

    iterations = []
    opinions = []

    # 各个迭代次数对应的观点值
    for iteration, opinions_data in data.items():
        iterations.append(iteration)
        # 存储观点数据
        opinions.append(opinions_data)

    # 绘制观点演化图
    for node, values in opinions[0].items():
        node_opinions = [opinions_data[node] for opinions_data in opinions]

        # 获取节点的位置索引
        node_index = list(opinions[0].keys()).index(node)

        # 根据行为值选择颜色
        color = '#6390eb' if true_labels[node_index] == 1 else '#fe6a6a'  # 根据位置索引选择颜色

        # 绘制观点演化图，并使用不同颜色
        ax.plot(iterations, node_opinions, color=color, linestyle='-')

        # 添加图例元素
        label = 'Node that Stays Online' if true_labels[node_index] == 1 else 'Node that Switches from Online to Offline'
        if label not in [leg.get_label() for leg in legend_elements]:
            legend_elements.append(Line2D([0], [0], color=color, label=label))
    
    # 在子图底部添加参数信息
    ax.text(0.5, -0.3, f'({idx+1})  (δ, λ, ε) = ({params[0]}, {params[1]}, {params[2]})',
            ha='center', va='center', fontsize=12, transform=ax.transAxes)


    # 只在第一列添加 y 轴标签
    if idx % num_cols == 0:
        ax.set_ylabel('Opinion', fontsize=12)

    # 只在最后一排添加 x 轴标签
    if idx // num_cols == num_rows - 1:
        ax.set_xlabel('Time', fontsize=12,labelpad=20)

    # 设置坐标轴刻度标签字体大小
    ax.tick_params(axis='both', which='major', labelsize=10)

    # 使用整数坐标
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))

# 添加图例，设置为一行放两个元素并放在图的正下方
fig.legend(handles=legend_elements, loc='lower center', fontsize=12, bbox_to_anchor=(0.5, -0.05), ncol=2)

# 隐藏剩余的子图（如果有的话）
if num_params < num_rows * num_cols:
    for ax in axes[num_params:]:
        ax.axis('off')

# 调整布局
plt.tight_layout()

# 保存图形为 PDF
plt.savefig('D:/研究生/论文/期刊论文/小论文1/草稿/图片/opinions_2_l1.eps', bbox_inches='tight')

plt.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

# 要筛选的特定参数组合
selected_params = [(0.2, 0.4, 0.1), (0.2, 0.4, 0.2), (0.2, 0.4, 0.3), (0.2, 0.4, 0.4), (0.2, 0.4, 0.5), (0.2, 0.4, 0.6), (0.2, 0.4, 0.7), (0.2, 0.4, 0.8), (0.2, 0.4, 0.9)]
# selected_params = [(0.3, 0.7, 0.1), (0.3, 0.7, 0.2), (0.3, 0.7, 0.3), (0.3, 0.7, 0.4), (0.3, 0.7, 0.5), (0.3, 0.7, 0.6), (0.3, 0.7, 0.7), (0.3, 0.7, 0.8), (0.3, 0.7, 0.9)]

# 提取特定参数组合的数据
selected_data = {params: merged_opinions_data[params] for params in selected_params}

# 提取真实标签
true_labels = window_labels_dict[14]

# 进行循环之前，创建一个空的图例列表
legend_elements = []

# 设置子图的规格为3x3
num_params = len(selected_params)
num_rows = 3
num_cols = 3
fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(15, 6))
axes = axes.flatten()  # 将子图的二维数组展平成一维数组，方便索引

# 遍历每个参数组合，并在对应的子图上绘制
for idx, (params, data) in enumerate(selected_data.items()):
    ax = axes[idx]

    iterations = []
    opinions = []

    # 各个迭代次数对应的观点值
    for iteration, opinions_data in data.items():
        iterations.append(iteration)
        # 存储观点数据
        opinions.append(opinions_data)

    # 绘制观点演化图
    for node, values in opinions[0].items():
        node_opinions = [opinions_data[node] for opinions_data in opinions]

        # 获取节点的位置索引
        node_index = list(opinions[0].keys()).index(node)

        # 根据行为值选择颜色
        color = '#6390eb' if true_labels[node_index] == 1 else '#fe6a6a'  # 根据位置索引选择颜色

        # 绘制观点演化图，并使用不同颜色
        ax.plot(iterations, node_opinions, color=color, linestyle='-')

        # 添加图例元素
        label = '保持在线的节点' if true_labels[node_index] == 1 else '在线转化为离线的节点'
        if label not in [leg.get_label() for leg in legend_elements]:
            legend_elements.append(Line2D([0], [0], color=color, label=label))
    
    # 在子图底部添加参数信息
    ax.text(0.5, -0.3, f'({idx+1})  (δ, λ, ε) = ({params[0]}, {params[1]}, {params[2]})',
            ha='center', va='center', fontsize=12, fontproperties=font_prop, transform=ax.transAxes)


    # 只在第一列添加 y 轴标签
    if idx % num_cols == 0:
        ax.set_ylabel('观点', fontsize=12, fontproperties=font_prop)

    # 只在最后一排添加 x 轴标签
    if idx // num_cols == num_rows - 1:
        ax.set_xlabel('时间', fontsize=12, fontproperties=font_prop,labelpad=20)

    # 设置坐标轴刻度标签字体大小
    ax.tick_params(axis='both', which='major', labelsize=10)

    # 使用整数坐标
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))

# 添加图例，设置为一行放两个元素并放在图的正下方
fig.legend(handles=legend_elements, loc='lower center', fontsize=12, bbox_to_anchor=(0.5, -0.05), ncol=2)

# 隐藏剩余的子图（如果有的话）
if num_params < num_rows * num_cols:
    for ax in axes[num_params:]:
        ax.axis('off')

# 调整布局
plt.tight_layout()

# 保存图形为 PDF
plt.savefig('D:/研究生/毕业论文/草稿/图片/opinions_3_l0.svg', bbox_inches='tight')

plt.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

# 要筛选的特定参数组合
# selected_params = [(0.2, 0.4, 0.1), (0.2, 0.4, 0.2), (0.2, 0.4, 0.3), (0.2, 0.4, 0.4), (0.2, 0.4, 0.5), (0.2, 0.4, 0.6), (0.2, 0.4, 0.7), (0.2, 0.4, 0.8), (0.2, 0.4, 0.9)]
selected_params = [(0.3, 0.7, 0.1), (0.3, 0.7, 0.2), (0.3, 0.7, 0.3), (0.3, 0.7, 0.4), (0.3, 0.7, 0.5), (0.3, 0.7, 0.6), (0.3, 0.7, 0.7), (0.3, 0.7, 0.8), (0.3, 0.7, 0.9)]

# 提取特定参数组合的数据
selected_data = {params: merged_opinions_data[params] for params in selected_params}

# 提取真实标签
true_labels = window_labels_dict[14]

# 进行循环之前，创建一个空的图例列表
legend_elements = []

# 设置子图的规格为3x3
num_params = len(selected_params)
num_rows = 3
num_cols = 3
fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(15, 6))
axes = axes.flatten()  # 将子图的二维数组展平成一维数组，方便索引

# 遍历每个参数组合，并在对应的子图上绘制
for idx, (params, data) in enumerate(selected_data.items()):
    ax = axes[idx]

    iterations = []
    opinions = []

    # 各个迭代次数对应的观点值
    for iteration, opinions_data in data.items():
        iterations.append(iteration)
        # 存储观点数据
        opinions.append(opinions_data)

    # 绘制观点演化图
    for node, values in opinions[0].items():
        node_opinions = [opinions_data[node] for opinions_data in opinions]

        # 获取节点的位置索引
        node_index = list(opinions[0].keys()).index(node)

        # 根据行为值选择颜色
        color = '#6390eb' if true_labels[node_index] == 1 else '#fe6a6a'  # 根据位置索引选择颜色

        # 绘制观点演化图，并使用不同颜色
        ax.plot(iterations, node_opinions, color=color, linestyle='-')

        # 添加图例元素
        label = 'Node that Stays Online' if true_labels[node_index] == 1 else 'Node that Switches from Online to Offline'
        if label not in [leg.get_label() for leg in legend_elements]:
            legend_elements.append(Line2D([0], [0], color=color, label=label))
    
    # 在子图底部添加参数信息
    ax.text(0.5, -0.3, f'({idx+1})  (δ, λ, ε) = ({params[0]}, {params[1]}, {params[2]})',
            ha='center', va='center', fontsize=12, transform=ax.transAxes)


    # 只在第一列添加 y 轴标签
    if idx % num_cols == 0:
        ax.set_ylabel('Opinion', fontsize=12)

    # 只在最后一排添加 x 轴标签
    if idx // num_cols == num_rows - 1:
        ax.set_xlabel('Time', fontsize=12,labelpad=20)

    # 设置坐标轴刻度标签字体大小
    ax.tick_params(axis='both', which='major', labelsize=10)

    # 使用整数坐标
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))

# 添加图例，设置为一行放两个元素并放在图的正下方
fig.legend(handles=legend_elements, loc='lower center', fontsize=12, bbox_to_anchor=(0.5, -0.05), ncol=2)

# 隐藏剩余的子图（如果有的话）
if num_params < num_rows * num_cols:
    for ax in axes[num_params:]:
        ax.axis('off')

# 调整布局
plt.tight_layout()

# 保存图形为 PDF
plt.savefig('D:/研究生/论文/期刊论文/小论文1/草稿/图片/opinions_3_l1.eps', bbox_inches='tight')

plt.show()

### 6.9.可视化dysat结果

In [ ]:
data_0 = pd.read_excel('D:/研究生/论文/毕业论文/代码/网络构建/直播平台网络模型/jupyter/处理数据/dysat_0.xlsx')
s1_0 = data_0['Loss']
s2_0 = data_0['Validation_accuracy']
s3_0 = data_0['Test_accuracy']
t = np.arange(len(s1_0))+1

data_1 = pd.read_excel('D:/研究生/论文/毕业论文/代码/网络构建/直播平台网络模型/jupyter/处理数据/dysat_1.xlsx')
s1_1 = data_1['Loss']
s2_1 = data_1['Validation_accuracy']
s3_1 = data_1['Test_accuracy']

In [ ]:
# 设置图像大小和样式
plt.figure(figsize=(8, 6), dpi=400)
plt.subplots_adjust(hspace=0.3)  # 调整子图之间的间距

# 子图 1：Loss 曲线
ax1 = plt.subplot(311)
ax1.plot(t, s1_0, color='#4169e0', label=r'$L_1$损失值', linewidth=2, marker='o')
ax1.plot(t, s1_1, color='#4169e0', label=r'$L_2$损失值', linewidth=2, marker='s')
ax1.grid(True, linestyle='--', alpha=0.6)
ax1.legend(fontsize=10)
plt.tick_params('x', labelbottom=False)

# 子图 2：Validation Accuracy 曲线
ax2 = plt.subplot(312, sharex=ax1)
ax2.plot(t, s2_0, color='#ff0000', label=r'$L_1$验证集AUC', linewidth=2, marker='o')
ax2.plot(t, s2_1, color='#ff0000', label=r'$L_2$验证集AUC', linewidth=2, marker='s')
ax2.set_ylabel("分数", fontsize=10.5)
ax2.grid(True, linestyle='--', alpha=0.6)
ax2.legend(fontsize=10)
plt.tick_params('x', labelbottom=False)

# 子图 3：Test Accuracy 曲线
ax3 = plt.subplot(313, sharex=ax1)
ax3.plot(t, s3_0, color='#00cdd0', label=r'$L_1$测试集AUC', linewidth=2, marker='o')
ax3.plot(t, s3_1, color='#00cdd0', label=r'$L_2$测试集AUC', linewidth=2, marker='s')
ax3.set_xlabel("窗口(无单位)", fontsize=10.5)
ax3.grid(True, linestyle='--', alpha=0.6)
ax3.legend(fontsize=10)

# 保存图形为 PDF
plt.savefig('D:/研究生/论文/毕业论文/代码/网络构建/直播平台网络模型/图片/dysat_l12.svg')

plt.show

In [ ]:
# 设置图像大小和样式
plt.figure(figsize=(8, 6), dpi=400)
plt.subplots_adjust(hspace=0.3)  # 调整子图之间的间距

# 子图 1：Loss 曲线
ax1 = plt.subplot(311)
ax1.plot(t, s1_0, color='#4169e0', label=r'$L_1$ Loss', linewidth=2, marker='o')
ax1.plot(t, s1_1, color='#4169e0', label=r'$L_2$ Loss', linewidth=2, marker='s')
ax1.grid(True, linestyle='--', alpha=0.6)
ax1.legend(fontsize=10)
plt.tick_params('x', labelbottom=False)

# 子图 2：Validation Accuracy 曲线
ax2 = plt.subplot(312, sharex=ax1)
ax2.plot(t, s2_0, color='#ff0000', label=r'$L_1$ Val_AUC', linewidth=2, marker='o')
ax2.plot(t, s2_1, color='#ff0000', label=r'$L_2$ Val_AUC', linewidth=2, marker='s')
ax2.set_ylabel("Score", fontsize=10.5)
ax2.grid(True, linestyle='--', alpha=0.6)
ax2.legend(fontsize=10)
plt.tick_params('x', labelbottom=False)

# 子图 3：Test Accuracy 曲线
ax3 = plt.subplot(313, sharex=ax1)
ax3.plot(t, s3_0, color='#00cdd0', label=r'$L_1$ Test_AUC', linewidth=2, marker='o')
ax3.plot(t, s3_1, color='#00cdd0', label=r'$L_2$ Test_AUC', linewidth=2, marker='s')
ax3.set_xlabel("Window", fontsize=10.5)
ax3.grid(True, linestyle='--', alpha=0.6)
ax3.legend(fontsize=10)

# 保存图形为 PDF
plt.savefig('D:/研究生/论文/期刊论文/小论文1/草稿/图片/dysat.eps')

plt.show()